In [1]:
import os
import re
import json
import zipfile
import requests
import numpy as np
import pandas as pd
from decimal import Decimal
# from tokens import YANDEX_ORG # Раскоменить для работы с API Яндекс.Организации и получения координат

In [2]:
def download(url)->str:
    response = requests.get(url) 
    if response.status_code == 200:
        content_disposition = response.headers.get("Content-Disposition")
        if content_disposition:
            filename = re.findall("filename=(.+)", content_disposition)
            if filename:
                filename = filename[0]
                filename = re.sub(r'"', '', filename)
                with open(filename, "wb") as file:
                    file.write(response.content)
                return filename
            else:
                return "Не удалось получить имя файла"
        else:
            return "Не удалось получить информацию о загруженном файле"
    else:
        return "Ошибка при загрузке файла"

    
def unzip(filename:str)->str:
    filenames = []
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        filenames = zip_ref.namelist()
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extract(filenames[0])
    return filenames[0]

# Школы

## Датасет с результатами ЕГЭ

[Ссылка на оригинальный датасет](https://data.mos.ru/odata/export/catalog?idFile=200291)



Набор данных «Результаты ЕГЭ» позволяет получить информацию об общих результатах государственной итоговой аттестации по программам среднего общего образования (далее – ГИА-11) каждой образовательной организации города Москвы.

В наборе представлены данные:

- об административных округах, районах, полных наименованиях образовательных организаций;

- о периоде, за который сдавался единый государственный экзамен (далее – ЕГЭ);

- о количестве участников, набравших не менее 200 баллов за 3 экзамена;

- о количестве участников, набравших более 160 баллов за 3 экзамена.

Результаты планируется публиковать ежегодно, таким образом можно будет проследить динамику повышения качества образования в Москве.


Современная ситуация в России

Согласно статье 59 Федерального закона от 29.12.2012 г. № 273-ФЗ «Об образовании в Российской Федерации», ГИА-11 завершает освоение основных образовательных программ среднего общего образования и является обязательной.

К ГИА-11 допускаются обучающиеся, не имеющие академической задолженности, в том числе за итоговое сочинение (изложение), и в полном объеме выполнившие учебный план или индивидуальный учебный план (имеющие годовые отметки по всем учебным предметам учебного плана за каждый год обучения по образовательной программе среднего общего образования не ниже удовлетворительных).

К ГИА-11 по учебным предметам, освоение которых завершилось ранее, допускаются обучающиеся X-XI (XII) классов, имеющие годовые отметки не ниже удовлетворительных по всем учебным предметам учебного плана за предпоследний год обучения.

Выпускники текущего года проходят ГИА-11 по двум обязательным учебным предметам (русский язык и математика) и учебным предметам по выбору.


Современная ситуация в Москве

Участники ГИА-11 могут узнать свои результаты по следующей схеме:

<table>
  <tr><td>обучающиеся образовательных организаций</td><td>в своей образовательной организации</td></tr>
  <tr><td>выпускники прошлых лет</td><td>в региональном центре обработки</td></tr>
</table>

На официальном Портале Мэра и Правительства Москвы www.mos.ru в разделе «Услуги и сервисы» можно оформить подписку на получение уведомлений о готовности результатов ГИА-11.

А знаете ли вы, что...

- Для проведения ГИА-11 предусмотрены следующие сроки: досрочный, основной и дополнительный периоды.

- Срок действия результатов ЕГЭ – 4 года, следующих за годом получения таких результатов.

- Результаты ЕГЭ каждого участника заносятся в федеральную информационную систему, бумажных свидетельств о результатах ЕГЭ не предусмотрено.


Материал подготовлен на основе информации data.mos.ru, dogm.mos.ru, mos.ru.

In [3]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=200291")
csvname = unzip(filename)

In [4]:
# датасет с результатами ЕГЭ, в нем нет адресов и нет координат, только названия школ
exams = pd.read_csv(csvname, sep=';')
exams = exams.drop(index=0) # удаляем первую строку с русским названием столбцов, английские ужесть в датасете
# удаляем пустой столбец по маске, т.к. его имя в часте нумерации меняется
exams = exams.drop(columns=[col for col in exams.columns if col.startswith("Unnamed:")])
# чистим имена школ от мусора, делаем все буквы маленькими, убираем кавычки, и др. найденные разночтения
exams['EDU_NAME'] = exams['EDU_NAME'].str.lower().str.strip().str.replace('«', '').str.replace('»', '') \
.str.replace('  ', ' ').str.replace('"', ' ').str.replace('общеобразовательн', 'образовательн').str.replace('№ ', '')
exams['geoData'] = None
exams.head()

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
1,8236,государственное бюджетное образовательное учре...,797946035,2016-2017,Восточный административный округ,район Богородское,5,19,None
2,8237,государственное казенное образовательное учреж...,797949150,2016-2017,Восточный административный округ,район Богородское,14,47,None
3,8238,государственное казенное образовательное учреж...,797949275,2016-2017,Восточный административный округ,район Вешняки,0,1,None
4,8239,частное учреждение образовательная организация...,797949425,2016-2017,Восточный административный округ,район Вешняки,0,6,None
5,8240,автономная некоммерческая организация средняя ...,797949570,2016-2017,Восточный административный округ,район Восточное Измайлово,1,5,None


In [5]:
exams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 1 to 4884
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                4884 non-null   object
 1   EDU_NAME          4884 non-null   object
 2   global_id         4884 non-null   object
 3   YEAR              4884 non-null   object
 4   AdmArea           4884 non-null   object
 5   District          4884 non-null   object
 6   PASSES_OVER_220   4884 non-null   object
 7   PASSER_UNDER_160  4884 non-null   object
 8   geoData           0 non-null      object
dtypes: object(9)
memory usage: 343.5+ KB


In [6]:
os.remove(filename)
os.remove(csvname)

## Датасет с координатами школ

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=217731)

Набор данных содержит информацию об образовательных организациях города Москвы: адреса, контактные телефоны, местоположение на карте, адреса интернет-сайтов учреждений, а также информацию о доступности зданий для лиц с ограниченными возможностями.

В соответствии с Федеральным Законом от 29.12.2012 № 273-ФЗ «Об образовании в Российской Федерации» (Закон) изменилась классификация образовательных организаций. Ранее образовательные организации классифицировались по «типам» и «видам» в зависимости от возраста обучающихся, в настоящее время согласно п.2 ст.10 Закона образование подразделяется на общее, профессиональное, дополнительное и профессиональное обучение, обеспечивающие возможность реализации права на образование в течение всей жизни (непрерывное образование).

Современные московские образовательные организации, как правило, реализуют несколько уровней общего образования (п.4 ст.10 Закона) – дошкольное, начальное общее, основное общее, среднее общее. Они также могут предлагать среднее и высшее профессиональное образование (п.5 ст.10 Закона) и четкой привязки образовательных организаций к прежней классификации (Детский сад → Средняя школа → Учреждение среднего профессионального образования → Высшее учебное заведение) уже нет.

У больших школ появились новые возможности - образовательные организации получили ресурс для повышения доступности реализуемых ими образовательных программ для московских семей. В зависимости от предпочтений семей школа предлагает дополнительное образование: организует работу кружков, секций, курсов различной направленности для детей и взрослых в любом из своих зданий.

Именно поэтому местом предоставления той или иной образовательной услуги должны считаться все здания, входящие в состав образовательной организации. Места оказания услуг общего и профессионального образования, а также – место работы конкретного кружка вы можете уточнить у администрации или на официальном сайте образовательной организации.

In [7]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=217731")
csvname = unzip(filename)

In [8]:
# датасет со списком школ от департамента образования, в нем координаты, но есть не все школы, например нет частных
schools = pd.read_csv(csvname, sep=';')
schools = schools.drop(index=0)
schools = schools.drop(columns=[col for col in schools.columns if col.startswith("Unnamed:")])
schools['FullName'] = schools['FullName'].str.lower().str.strip().str.replace('«', '').str.replace('»', '') \
.str.replace('  ', ' ').str.replace('"', ' ').str.replace('общеобразовательн', 'образовательн').str.replace('№ ', '')
# Преобразование значения в колонке 'geoData' в список координат
# schools['geoData'] = schools['geoData'].str.replace('{coordinates=[', '').str.replace('], type=Polygon}', '')
schools.head()

,OrgType,ReorganizationStatus,IDEKIS,FullName,ShortName,Number,INN,KPP,OGRN,LegalOrganization,...,WebSite,EducationPrograms,EducationalServices,LicensingAndAccreditation,SupplementaryClassesAndGroups,NumberofStudentsInOO,TheContingentOfPreschoolersStudyingOO,global_id,geoData,geodata_center
1,прочее,действующее,14500,государственное автономное учреждение города м...,ГАУ Медиацентр,NaN,7718924940,771801001,1137746210161,автономное,...,shkolamoskva.ru,NaN,nested data,nested data,nested data,nested data,nested data,1139421628,"{coordinates=[[[37.704597369, 55.80046703], [3...",NaN
2,организация дополнительного профессионального ...,действующее,14494,государственное автономное образовательное учр...,ГАОУ ДПО «Корпоративный университет»,NaN,7714239823,771301001,1027700566761,автономное,...,corp-univer.ru,Профессиональное образование | высшее образова...,nested data,nested data,nested data,nested data,nested data,2386776971,"{coordinates=[[[[37.539544671, 55.799821354], ...",NaN
3,организация дополнительного профессионального ...,действующее,14506,государственное автономное образовательное учр...,ГАОУ ДПО МЦРПО,NaN,7707329480,772201001,1057707018192,автономное,...,mcrpo.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2645605456,"{coordinates=[[[[37.711001873, 55.762921204], ...",NaN
4,организация дополнительного профессионального ...,действующее,14605,государственное бюджетное образовательное учре...,ГБОУ ДПО МЦВП,NaN,7723029678,772301001,1027739293944,бюджетное,...,mcvp.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2645605458,"{coordinates=[[[[37.65201246, 55.742196123], [...",NaN
5,организация дополнительного профессионального ...,действующее,13070,государственное автономное образовательное учр...,ГАОУ ДПО ЦПМ,NaN,7725618950,770401001,1077761137222,автономное,...,cpm.dogm.mos.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2654965921,"{coordinates=[[[[37.56961301, 55.671061893], [...",NaN


In [9]:
schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 1 to 578
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   OrgType                                578 non-null    object
 1   ReorganizationStatus                   578 non-null    object
 2   IDEKIS                                 578 non-null    object
 3   FullName                               578 non-null    object
 4   ShortName                              578 non-null    object
 5   Number                                 475 non-null    object
 6   INN                                    578 non-null    object
 7   KPP                                    578 non-null    object
 8   OGRN                                   578 non-null    object
 9   LegalOrganization                      578 non-null    object
 10  Subordination                          578 non-null    object
 11  ChiefName          

In [10]:
os.remove(filename)
os.remove(csvname)

## Координаты из списка школ в результаты ЭГЕ

In [11]:
exams['geoData'] = exams['EDU_NAME'].apply(lambda x: schools.loc[schools['FullName'] == x, 'geoData'].values[0] if x in schools['FullName'].values else None)
exams.head()

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
1,8236,государственное бюджетное образовательное учре...,797946035,2016-2017,Восточный административный округ,район Богородское,5,19,None
2,8237,государственное казенное образовательное учреж...,797949150,2016-2017,Восточный административный округ,район Богородское,14,47,"{coordinates=[[[[37.710083127, 55.825085846], ..."
3,8238,государственное казенное образовательное учреж...,797949275,2016-2017,Восточный административный округ,район Вешняки,0,1,"{coordinates=[[[[37.807984129, 55.726758653], ..."
4,8239,частное учреждение образовательная организация...,797949425,2016-2017,Восточный административный округ,район Вешняки,0,6,None
5,8240,автономная некоммерческая организация средняя ...,797949570,2016-2017,Восточный административный округ,район Восточное Измайлово,1,5,None


In [12]:
# Загрузка кэша из файла, если файл существует
YANDEX_ORG = 0 # Закоментить строку если есть ключ
cache_file = 'geo_cache.json'
if os.path.exists(cache_file):
    with open(cache_file, 'r') as f:
        geo_cache = json.load(f)
else:
    geo_cache = {}

# Функция для запроса к API Яндекса
def get_geo_data(row):
    if pd.notna(row['geoData']):
        return row['geoData']
    
    key = f"{row['AdmArea']}_{row['District']}_{row['EDU_NAME']}"
    if key in geo_cache:
        return geo_cache[key]
    
    url = "https://search-maps.yandex.ru/v1/"
    params = {
        "apikey": YANDEX_ORG,
        "lang": "ru_RU",
        "text": f"Москва, {row['AdmArea']}, {row['District']}, {row['EDU_NAME']}"
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'features' in data and len(data['features']) > 0:
            geo_data = data['features'][0]['geometry']
            geo_cache[key] = geo_data
            return geo_data
        else:
            geo_cache[key] = None
            return None
    except Exception as e:
        print(f"Ошибка при запросе к API: {e}")
        geo_cache[key] = None
        return None

# Применение функции к строкам DataFrame
exams['geoData'] = exams.apply(get_geo_data, axis=1)

# Сохранение кэша в файл
with open(cache_file, 'w', encoding='utf-8') as f:
    json.dump(geo_cache, f, ensure_ascii=False, indent=4)

In [13]:
pd.set_option('display.max_colwidth', None)
exams[exams['geoData'].isna()]

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
574,8810,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,798000274,2016-2017,Юго-Восточный административный округ,Нижегородский район,5,22,None
1195,9432,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,880632637,2017-2018,Юго-Восточный административный округ,Нижегородский район,4,21,None
1525,9760,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,1011905093,2018-2019,Юго-Восточный административный округ,Нижегородский район,8,31,None


In [14]:
# Ручное добавление одной школы"
exams.loc[exams['EDU_NAME'] == "государственное бюджетное образовательное учреждение города москвы школа 439 интеллект", 'geoData'] = "{'type': 'Point', 'coordinates': [37.745376, 55.731212]}"
exams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 1 to 4884
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                4884 non-null   object
 1   EDU_NAME          4884 non-null   object
 2   global_id         4884 non-null   object
 3   YEAR              4884 non-null   object
 4   AdmArea           4884 non-null   object
 5   District          4884 non-null   object
 6   PASSES_OVER_220   4884 non-null   object
 7   PASSER_UNDER_160  4884 non-null   object
 8   geoData           4884 non-null   object
dtypes: object(9)
memory usage: 343.5+ KB


In [15]:
exams.to_csv('exams.csv', index=False, encoding='utf-8')

# Экология

## Датасет загрязнений атмосферы

[Ссылка на оригинальный датасет](https://data.mos.ru/odata/export/catalog?idFile=218714)

Набор данных позволяет получить информацию о месте отбора проб атмосферного воздуха, дате проведения отбора, основных результатах исследования. Для удобства ориентирования предусмотрена возможность просмотра местоположения контрольной точки исследования атмосферного воздуха на карте города Москвы.

 

В 2015 году Министерством природных ресурсов и экологии РФ был составлен экологический рейтинг российских городов, при составлении которого учитывались данные о состоянии атмосферного воздуха, водопользования, по выбросам от транспорта, об использовании городских территорий и энергопотреблении. По результатам рейтинга Москва - лидер по экологическому развитию среди остальных российских городов.

 

В городе Москве организована уникальная система мониторинга загрязнения атмосферного воздуха, в состав которой входят, кроме стационарных станций контроля загрязнения атмосферного воздуха, три передвижные лаборатории ГПБУ «Мосэкомониторинг».

 

Передвижная лаборатория – это автомобиль, оснащенный современным оборудованием, которое позволяет измерять загрязняющие вещества в режиме реального времени по основным загрязняющим веществам и отбирать пробы воздуха для последующего анализа в лаборатории на расширенный перечень веществ.

 

В Российской Федерации установлены гигиенические нормативы предельно допустимых концентраций (ПДК) загрязняющих веществ в атмосферном воздухе. В случае выявления в ходе рейдов передвижной лаборатории превышений нормативов информация о зафиксированных нарушениях направляется в соответствующие органы исполнительной власти для принятия мер.

 

А ЗНАЕТЕ ЛИ ВЫ, ЧТО?

 

Много сил, труда и терпения   потребовалось ученым, чтобы доказать, что воздух - не самостоятельное вещество, а сложная смесь газов. Впервые высказался о сложном составе воздуха ученый-художник Леонардо да Винчи (XV век).

Количественный состав воздуха  установил французский ученый Антуан Лоран Лавуазье. По результатам своих опытов он сделал вывод, что воздух в целом состоит из кислорода, пригодного для дыхания и горения, и азота в пропорциях 1/5 и 4/5 объема соответственно.

 

В настоящее время известно, что атмосферный воздух – это смесь газов, состоящая из постоянных и переменных компонентов. На высотах до 100 км относительный состав газов составляет: азот 78%, кислород 21%, около 1% инертного газа аргона, вклад остальных постоянных компонентов существенно меньше.

 

К переменным компонентам относится водяной пар, озон, двуокись углерода, диоксид серы, диоксид азота и другие, их доля составляет сотые и тысячные доли процентов и они называются «малыми» газами. Кроме этого, в атмосфере присутствуют взвешенные твердые частицы и аэрозоли.

 

Изучение загрязнения атмосферного воздуха началось с развитием промышленности и ростом городов.

 

В России одним из первых на вред промышленных выбросов в 1851 году обратил внимание Д.И.Менделеев в статье «О происхождении дыма», в которой писал: «Дым затемняет день, проникает в жилище, грязнит фасады зданий и общественные фабрики, причиняет многие неудобства и нездоровья».

 

Во второй половине XIX века в таком крупном индустриальном городе, как Чикаго началась борьба с дымом, в рамках которой появление густого дыма было объявлено нарушением общественного порядка, за которое причитался штраф, а с 1928 года в Чикаго существует сеть станций для наблюдения за загрязнением воздуха.

 

В СССР первые исследования по гигиене атмосферного воздуха проводились в 1929-1930 годах. Их целью было получить научно обоснованные материалы о степени и характере загрязнения атмосферного воздуха в связи с развитием и реконструкцией городов и строительством новых предприятий. Полученные данные использовались при разработке первых элементов санитарного законодательства о размещении промышленных предприятий и принципов организации санитарно-защитных зон между жилыми районами и предприятиями.

 

Постановление Совета Министров СССР от 29 мая 1949 г. «О мерах борьбы с загрязнением атмосферного воздуха и об улучшении санитарно-гигиенических условий населенных мест» положило начало проведению организованных мероприятий по охране атмосферного воздуха в масштабе всей страны, началось активное развитие законодательства об охране атмосферного воздуха.

 

На Конференции ООН по проблемам окружающей среды (Стокгольм, 1972 год) в рамках декларации был принят принцип о том, что природные ресурсы Земли, включая воздух, должны быть сохранены на благо нынешнего и будущих поколений путем тщательного планирования и управления по мере необходимости.

 

Стокгольмская конференция положила начало быстрому развитию национальных и международных организаций, направляющих и координирующих деятельность по сохранению окружающей среды. Если до Стокгольмской конференции государственные органы по охране окружающей среды были только в Швеции, США, Канаде, Великобритании и Японии, то после неё подобные структуры были организованы практически во всех государствах, в том числе в СССР.

 

В настоящее время на территории Российской Федерации действуют федеральные законы «Об охране окружающей среды» и «Об охране атмосферного воздуха», которые устанавливают правовые основы охраны атмосферного воздуха.

 

Материал подготовлен на основе информации: http://data.mos.ru/,  http://www.mosecom.ru/, http://www.meteorf.ru/, http://www.mnr.gov.ru/, http://www.ecoethics.ru/, http://energetika.in.ua/.


In [16]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=217731")
csvname = unzip(filename)

In [17]:
# датасет с указанием координаты загрязнений воздуха
ecology_air = pd.read_csv(csvname, sep=';')
ecology_air = ecology_air.drop(index=0)
ecology_air = ecology_air.drop(columns=[col for col in ecology_air.columns if col.startswith("Unnamed:")])
ecology_air.head()

,OrgType,ReorganizationStatus,IDEKIS,FullName,ShortName,Number,INN,KPP,OGRN,LegalOrganization,...,WebSite,EducationPrograms,EducationalServices,LicensingAndAccreditation,SupplementaryClassesAndGroups,NumberofStudentsInOO,TheContingentOfPreschoolersStudyingOO,global_id,geoData,geodata_center
1,прочее,действующее,14500,Государственное автономное учреждение города Москвы «Медиацентр»,ГАУ Медиацентр,NaN,7718924940,771801001,1137746210161,автономное,...,shkolamoskva.ru,NaN,nested data,nested data,nested data,nested data,nested data,1139421628,"{coordinates=[[[37.704597369, 55.80046703], [37.705485479, 55.800596013], [37.705559984, 55.800425152], [37.70563151, 55.800433527], [37.70589377, 55.79984388], [37.705771581, 55.799827129], [37.706009999, 55.799250874], [37.705658332, 55.79921067], [37.705402032, 55.799820428], [37.705554024, 55.79984053], [37.705524222, 55.799912561], [37.705136792, 55.799863982], [37.705014602, 55.800126979], [37.705413953, 55.800190634], [37.705327526, 55.800371548], [37.704963467, 55.800319428], [37.704975388, 55.800284809], [37.704886452, 55.80027104], [37.70486857, 55.800309568], [37.704677836, 55.800287791], [37.704597369, 55.80046703]]], type=Polygon}",NaN
2,организация дополнительного профессионального образования,действующее,14494,Государственное автономное образовательное учреждение дополнительного профессионального образования города Москвы «Корпоративный университет московского образования»,ГАОУ ДПО «Корпоративный университет»,NaN,7714239823,771301001,1027700566761,автономное,...,corp-univer.ru,"Профессиональное образование | высшее образование - бакалавриат # Профессиональное образование | высшее образование - специалитет, магистратура # Профессиональное образование | высшее образование - подготовка кадров высшей квалификации # Дополнительное образование | дополнительное образование детей # Дополнительное образование | дополнительное профессиональное образование # Дополнительное образование | дополнительное образование взрослых",nested data,nested data,nested data,nested data,nested data,2386776971,"{coordinates=[[[[37.539544671, 55.799821354], [37.53935791, 55.799897294], [37.53935791, 55.799975467], [37.539777129, 55.800289276], [37.539975812, 55.800201052], [37.539979785, 55.800139631], [37.539850642, 55.800040239], [37.539940049, 55.799996686], [37.539793024, 55.799887243], [37.539693683, 55.79993303], [37.539544671, 55.799821354]]], [[[37.560320716, 55.822874392], [37.560333905, 55.822858568], [37.560420206, 55.822748478], [37.56042418, 55.822673698], [37.560104302, 55.822584409], [37.560072512, 55.822634634], [37.560068539, 55.822653608], [37.560112249, 55.822669234], [37.559803149, 55.823045713], [37.559746525, 55.823031762], [37.559696854, 55.823077522], [37.559698841, 55.82309538], [37.559836925, 55.823132211], [37.559843879, 55.823189132], [37.560016732, 55.823232659], [37.560132961, 55.823104308], [37.560132961, 55.823047387], [37.560187599, 55.822975957], [37.560478668, 55.823051294], [37.560541253, 55.822973167], [37.560542247, 55.822953077], [37.560267072, 55.822881646], [37.56028098, 55.822865463], [37.560320716, 55.822874392]]], [[[37.618387714, 55.741878447], [37.618266121, 55.742030776], [37.618269697, 55.742087144], [37.618862167, 55.742235447], [37.618994489, 55.742067684], [37.618831173, 55.742023394], [37.618825212, 55.741984473], [37.618387714, 55.741878447]]], [[[37.557159683, 55.765908679], [37.556898416, 55.766061808], [37.556891462, 55.766096458], [37.557064315, 55.766190347], [37.557332536, 55.766034424], [37.557332536, 55.766003127], [37.557159683, 55.765908679]]], [[[37.592391991, 55.74329142], [37.592391991, 55.743347899], [37.592455569, 55.743503354], [37.592521134, 55.743497203], [37.592520141, 55.743483223], [37.592718823, 55.743459737], [37.592716836, 55.743398785], [37.59307049, 55.743360201], [37.593091352, 55.743421153], [37.593273146, 55.743396548], [37.593279107, 55.743412206], [37.593360566, 55.74340214], [37.59336156, 55.743351813], [37.593304935, 55.74

In [18]:
os.remove(filename)
os.remove(csvname)

In [19]:
ecology_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 1 to 578
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   OrgType                                578 non-null    object
 1   ReorganizationStatus                   578 non-null    object
 2   IDEKIS                                 578 non-null    object
 3   FullName                               578 non-null    object
 4   ShortName                              578 non-null    object
 5   Number                                 475 non-null    object
 6   INN                                    578 non-null    object
 7   KPP                                    578 non-null    object
 8   OGRN                                   578 non-null    object
 9   LegalOrganization                      578 non-null    object
 10  Subordination                          578 non-null    object
 11  ChiefName          

## Датасет шума


[Офстраница](https://data.mos.ru/opendata/2449)

[Ссылка на датасет](https://data.mos.ru/odata/export/catalog?idFile=203926)


Набор данных позволяет получить информацию о месте проведения измерений уровней шума, источнике шума, дате и основных результатах исследования, о наличии/отсутствии превышений установленных нормативов предельно допустимых уровней шума, а также имеется возможность посмотреть местоположение проведенных измерений на карте города Москвы.

Повышенная шумовая нагрузка характерна для всех мегаполисов мира, и Москва не является исключением.

Основными источниками шумового воздействия на территории Москвы являются автотранспорт, железнодорожный транспорт, авиатранспорт, промышленные предприятия, строительные площадки.

Организацией, осуществляющей проведение государственного экологического мониторинга уровней шума на территории города Москвы, является подведомственное Департаменту природопользования государственное природоохранное бюджетное учреждение «Мосэкомониторинг». На территории города Москвы измерения уровней шума проводятся специализированной акустической службой ГПБУ «Мосэкомониторинг» как в дневное, так и в ночное время суток.

В Российской Федерации установлены санитарные нормы предельно допустимого уровня шума для различных городских территорий. В случае превышения нормативов допустимого уровня шума информация направляется в соответствующие органы исполнительной власти для принятия мер.

А ЗНАЕТЕ ЛИ ВЫ, ЧТО?

Человек всегда жил в мире звуков и шума. Шум является неотъемлемой частью городской жизни, так что каждый житель мегаполиса в той или иной степени подвергается воздействию повышенного шума в течение всего дня.

По данным Всемирной организации здравоохранения, высокие уровни шума могут вызвать такие негативные последствия для здоровья, как повышенная раздражимость, нарушения сна, ухудшение психического здоровья, поведенческие эффекты, снижение работоспособности и даже заболевания сердечно-сосудистой системы. При этом люди наиболее чувствительны к шуму в ночное время.

Однако дискомфорт вызывает не только шумовое загрязнение, но и полное отсутствие шума. Более того, звуки определённой силы повышают работоспособность и стимулируют процесс мышления (в особенности процесс счёта) и, наоборот, при полном отсутствии шумов человек теряет работоспособность и испытывает стресс.

Самое тихое место в мире согласно Книге рекордов Гиннеса находится в Южном Миннеаполисе в Лабораториях Орфилда (штат Миннесота, США), безэховой тестовой камере, блокирующей 99,99% внешних звуков, и никто еще не смог находится в этой комнате более 45 минут.

Первые в мире нормы по ограничению шума  «Временные санитарные нормы и правила по ограничению шума на производстве» были разработаны в СССР и утверждены Министерством здравоохранения СССР в 1956 году в качестве обязательных для всех заводов и проектных организаций. Эти нормативы первоначально разрабатывались для определения условий вредности труда, а впоследствии на основании опыта применения первых норм по ограничению шума, рекомендаций Международной организации по стандартизации и большого числа работ, посвященных изучению различных аспектов влияния шума на человека, Министерством здравоохранения СССР были также утверждены санитарные нормы в отношении территории поселений.

Однако о неблагоприятном воздействии шума на живые организмы было известно еще задолго до появления санитарных норм по шуму и возникновения таких наук, как акустика и аудиология. На Руси, когда стерлядь входила в малые реки на нерест, церкви не звонили в колокола.

В настоящее время на территории Российской Федерации установлены нормативы допустимого уровня шума для различных городских территорий. Так, территории, непосредственно прилегающие к жилым домам, зданиям поликлиник и амбулаторий, домам отдыха и пансионатов, не должны превышать 55 дБА по эквивалентному и 70 дБА по максимальному уровням звука, для ночного времени суток допустимый уровень шума снижается на 10 дБА.

Материал подготовлен на основе информации: 

http://web-local.rudn.ru/, http://www.trudcontrol.ru/, http://mirfactov.com/, http://data.mos.ru.


In [20]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=203926")
csvname = unzip(filename)

In [21]:
ecology_noise = pd.read_csv(csvname, sep=';')
ecology_noise = ecology_noise.drop(index=0)
ecology_noise = ecology_noise.drop(columns=[col for col in ecology_noise.columns if col.startswith("Unnamed:")])
ecology_noise.head()

,global_id,ID,Date,AdmArea,District,Longitude_WGS84,Latitude_WGS84,Location,TerritoryType,NoiseCategory,Results,geoData,geodata_center
1,279137532,982,06.09.2016,Южный административный округ,Даниловский район,37.634479,55.708770,"Даниловская набережная, дом 2, корпус 1","Территория, прилегающая к жилым домам первой линии застройки от автотрасс",Строительные работы,Выявлены превышения нормативов допустимого уровня шума от автотранспорта.,"{coordinates=[37.634479, 55.70877], type=Point}","{coordinates=[37.634479, 55.70877], type=Point}"
2,279137533,1000,07.09.2016,Юго-Восточный административный округ,район Некрасовка,37.927329,55.708658,"Сочинская улица, дом 2","Территория, прилегающая к жилым домам первой линии застройки от автотрасс",Автотранспорт,Выявлены превышения нормативов допустимого уровня шума от автотранспорта.,"{coordinates=[37.927329, 55.708658], type=Point}","{coordinates=[37.927329, 55.708658], type=Point}"
3,279137534,1001,08.09.2016,Юго-Восточный административный округ,район Люблино,37.751071,55.687259,"Краснодонская улица, дом 2, корпус 3","Территория, прилегающая к жилым домам за первой линией застройки от автотрасс","[Кафе, ресторан, клуб, Музыка]","Превышения нормативов допустимого уровня шума от работы летнего кафе не выявлены, выявлены превышения нормативов допустимого уровня шума от автотранспорта.","{coordinates=[37.751071, 55.687259], type=Point}","{coordinates=[37.751071, 55.687259], type=Point}"
4,279137535,1002,09.09.2016,Юго-Восточный административный округ,район Текстильщики,37.737426,55.705239,"8-я улица Текстильщиков, дом 2, корпус 1","Территория, прилегающая к жилым домам первой линии застройки от автотрасс",Генераторная установка,Выявлены превышения нормативов допустимого уровня шума от автотранспорта.,"{coordinates=[37.737426, 55.705239], type=Point}","{coordinates=[37.737426, 55.705239], type=Point}"
5,279137536,1003,09.09.2016,Юго-Восточный административный округ,район Лефортово,37.680670,55.755616,"Волочаевская улица, дом 14, корпус 1","Территория, прилегающая к жилым домам за первой линией застройки от автотрасс",Вентиляционные системы,Выявлены превышения нормативов допустимого уровня шума от работы вентиляционного оборудования.,"{coordinates=[37.68067, 55.755616], type=Point}","{coordinates=[37.68067, 55.755616], type=Point}"


In [22]:
os.remove(filename)
os.remove(csvname)

In [23]:
ecology_noise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13015 entries, 1 to 13015
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   global_id        13015 non-null  object
 1   ID               13015 non-null  object
 2   Date             13015 non-null  object
 3   AdmArea          13015 non-null  object
 4   District         13015 non-null  object
 5   Longitude_WGS84  13015 non-null  object
 6   Latitude_WGS84   13015 non-null  object
 7   Location         13015 non-null  object
 8   TerritoryType    13015 non-null  object
 9   NoiseCategory    13015 non-null  object
 10  Results          13015 non-null  object
 11  geoData          13015 non-null  object
 12  geodata_center   13015 non-null  object
dtypes: object(13)
memory usage: 1.3+ MB


## Датасет загрязнения почвы

[офисточник](https://data.mos.ru/opendata/2452)

[датасет](https://data.mos.ru/odata/export/catalog?idFile=218873)

Набор данных содержит результаты ежегодного экологического мониторинга почв в городе Москве за 2015 год.

На Портале размещены данные о качестве почв на 253 площадках постоянного мониторинга, расположенных на территории города Москвы. Для удобства ориентирования предусмотрена возможность просмотра местоположения площадок постоянного мониторинга на карте города Москвы.

Сеть мониторинга за состоянием почв города Москвы включает в себя более 1300 площадок постоянного мониторинга, ежегодно из которых обследуется порядка 200-300. Наблюдательная сеть сформирована с учетом территориального деления (охватывает все административные округа города Москвы) и функционального зонирования (зоны ООПТ, природных и озелененных территорий, жилые, общественные, производственные зоны, а также зоны транспортной инфраструктуры), что позволяет получать максимально полную структурированную информацию о современном состоянии почвенного покрова в городе, отслеживать тенденции изменения состояния почв и выявлять наиболее актуальные проблемы в данной области.

Оценка качества почв города Москвы на основании суммарного показателя загрязнения Zc проводится по содержанию тяжелых металлов: цинка, кадмия, свинца, мышьяка, ртути (вещества I класса опасности), никеля, меди (вещества II класса опасности).

По результатам ежегодного экологического мониторинга 2015 года установлено, что почвы большей части территории города Москвы (более 94 %) характеризуются минимальными уровнями загрязнения (Zc < 16).

А ЗНАЕТЕ ЛИ ВЫ, ЧТО?

Продовольственной и сельскохозяйственной организацией ООН (ФАО) 2015 год был объявлен Международным годом почв.

Генеральный директор ФАО Жозе Грациану да Силва заявил: «Разнообразие ролей, которые играют почвы в нашей жизни, зачастую остается незамеченным. Почвы не умеют говорить, и мало кто выступает в их защиту».

В.И.Вернадский отнес почву к особым биокосным телам природы, которые занимают промежуточное положение между живыми организмами и косными телами (горные породы, минералы).

Почва является вторым после океанов крупнейшим хранилищем углерода на планете (в ней содержится до 75 % углерода). Естественная способность почвы сохранять углерод имеет важное значение для смягчения и адаптации к изменению климата, а также для повышения устойчивости к наводнениям и засухе.

Почва - главный регулятор состава атмосферы Земли. Обусловлено это деятельностью почвенных микроорганизмов, в огромных количествах производящих разнообразные газы - азот и его окислы, кислород, диоксид и оксид углерода, метан и другие углеводороды, сероводород, ряд прочих летучих соединений.

Ученые обнаружили, что почва является одним из наших крупнейших резервуаров биоразнообразия, содержащих почти треть всей жизни планеты! Чайная ложка почвы может быть домом для миллиардов микробов. И жизнь, что рождается в почве, составляет сложную сеть взаимосвязей.

Микроорганизмы, которые живут в почве, используются в медицине для лечения заболеваний. Почти все антибиотики, которые помогают нам бороться с инфекциями, происходят из почвы. Эти организмы дали нам много жизненно необходимых лекарств, таких как антибиотик стрептомицин и препарат циклоспорин, используемый для предотвращения отторжения трансплантатов.

В городах почвы выполняют важные экологические функции: обеспечивают условия для произрастания зеленых насаждений, накапливают в толще загрязняющие вещества и удерживают их от проникновения в грунтовые воды. Биоразнообразие почв исключительно важно для поддержания почвенных функций и, таким образом, экосистемных товаров и услуг, связанных с почвами.

Материалы подготовлены на основе информации: «Методические рекомендации по определению степени загрязнения городских почв и грунтов и проведению инвентаризации территорий, требующих рекультивации» (Москва, 2004), Методические указания МУ 2.1.7.730-99 «Гигиеническая оценка качества почвы населенных мест», СанПиН 4266-87 Методические указания по оценке степени опасности загрязнения почвы химическими веществами, http://ru-ecology.info.


In [24]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=218873")
csvname = unzip(filename)

In [25]:
ecology_dirt = pd.read_csv(csvname, sep=';')
ecology_dirt = ecology_dirt.drop(index=0)
ecology_dirt = ecology_dirt.drop(columns=[col for col in ecology_dirt.columns if col.startswith("Unnamed:")])
ecology_dirt.head()

,global_id,ID,Year,AdmArea,District,Address,EstimatedindexValue,PollutionCategory,PollutionLevel,geoData,geodata_center
1,477298795,256,2016,Зеленоградский административный округ,район Крюково,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Крюково, город Зеленоград, корпус 1606",0,допустимая,минимальный,"{coordinates=[37.150715207, 55.976782064], type=Point}","{coordinates=[37.150715207, 55.976782064], type=Point}"
2,477298796,257,2016,Зеленоградский административный округ,район Силино,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Силино, город Зеленоград, корпус 1140",0,допустимая,минимальный,"{coordinates=[37.171732969, 55.994017593], type=Point}","{coordinates=[37.171732969, 55.994017593], type=Point}"
3,477298797,258,2016,Зеленоградский административный округ,район Савёлки,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Савелки, город Зеленоград, корпус 623",1.5,допустимая,минимальный,"{coordinates=[37.227688881, 55.990404298], type=Point}","{coordinates=[37.227688881, 55.990404298], type=Point}"
4,477298798,259,2016,Зеленоградский административный округ,район Матушкино,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Матушкино, город Зеленоград, Панфиловский проспект, дом 10",1.7,допустимая,минимальный,"{coordinates=[37.202221608, 56.012200797], type=Point}","{coordinates=[37.202221608, 56.012200797], type=Point}"
5,477298799,260,2016,Северо-Западный административный округ,район Покровское-Стрешнево,"город Москва, Сосновая аллея, дом 1, строение 1",0,допустимая,минимальный,"{coordinates=[37.461418684, 55.819693833], type=Point}","{coordinates=[37.461418684, 55.819693833], type=Point}"


In [26]:
os.remove(filename)
os.remove(csvname)

In [27]:
ecology_dirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 1 to 2442
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   global_id            2442 non-null   object
 1   ID                   2442 non-null   object
 2   Year                 2442 non-null   object
 3   AdmArea              2442 non-null   object
 4   District             2442 non-null   object
 5   Address              2442 non-null   object
 6   EstimatedindexValue  2442 non-null   object
 7   PollutionCategory    2442 non-null   object
 8   PollutionLevel       2442 non-null   object
 9   geoData              2442 non-null   object
 10  geodata_center       2442 non-null   object
dtypes: object(11)
memory usage: 210.0+ KB


## Датасет среднемесячного состояния воздуха

[Офстраница](https://data.mos.ru/opendata/2453)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215804)

Набор данных содержит информацию о средних за месяц концентрациях загрязняющих веществ в атмосферном воздухе Москвы, измеряемых автоматическими станциями контроля загрязнения атмосферы (АСКЗА).

Датасет позволяет получить информацию в разрезе каждой отдельной АСКЗА о средних за месяц концентрациях загрязняющих веществ в абсолютных единицах и в сравнении с предельно допустимыми среднесуточными концентрациями.

Основу мониторинга атмосферного воздуха города Москвы составляет сеть стационарных АСКЗА, которые в круглосуточном непрерывном режиме (с периодичностью один раз в двадцать минут) измеряют содержание в атмосферном воздухе загрязняющих веществ, характерных для выбросов автотранспорта и промышленных предприятий города Москвы.

Сеть автоматических станций мониторинга атмосферного воздуха включает в себя стационарные станции, место размещение которых сохраняется постоянно на протяжении долгого времени, и мобильные станции, размещаемые временно для решения локальных задач (выявление источника загрязнения, определение эффективности природоохранных мероприятий и т.д.).

Датасет включает информацию по стационарным АСКЗА, постоянность размещения которых на одной территории позволяет контролировать динамику содержания загрязняющих веществ в долговременной перспективе.

При ознакомлении с данными датасета важно учитывать, что АСКЗА в системе мониторинга атмосферного воздуха города Москвы ориентированы на контроль загрязнения воздуха на территориях различного функционального назначения: территории вблизи автомагистралей, жилые территории, находящиеся под влиянием смешанных антропогенных источников загрязнений; природные территории, «новая» Москва.

Перечень загрязняющих веществ на АСКЗА определяется функциональным назначением контролируемой территории. Наибольшее количество загрязняющих веществ измеряется вблизи автотрасс и на территориях, находящихся под воздействием очистных сооружений и нефтеперерабатывающего завода.

В общем, список измеряемых на станции загрязняющих веществ может широко варьироваться от 3-х основных параметров (оксид углерода, диоксид и оксид азота) до 10, включая специфические, такие как сероводород (индикатор выбросов нефтехимических производств и эмиссии очистных сооружений канализации).

Среднемесячные концентрации контролируемых параметров представлены как в абсолютных единицах (мг/м3), так и в долях ПДКсс (предельно допустимая среднесуточная концентрация химического вещества в воздухе населённых мест). Концентрация кислорода (О2) выражена в процентной плотности (%).

 

А ЗНАЕТЕ ЛИ ВЫ, ЧТО?

Благодаря реализуемым природоохранным мероприятиям, долгосрочная динамика демонстрирует постепенное снижение содержания загрязняющих веществ в атмосферном воздухе Москвы, в том числе оксида углерода и оксида азота. В первую очередь, это обусловлено существенным обновлением автопарка города, произошедшим, в том числе, из-за введения ограничения на въезд грузового транспорта и автобусов низких экологических классов, введением требований к качеству моторного топлива и другими крупномасштабными природоохранными мероприятиями, реализуемыми на крупных предприятиях.

Следует отметить, что существенное влияние на уровень загрязнения атмосферного воздуха оказывают метеорологические условия (в том числе слабый ветер, отсутствие осадков), которые приводят к повышению уровня загрязнения атмосферного воздуха. Как правило, такие условия отмечаются преимущественно в ночные и утренние часы.

Сравнительный анализ содержания загрязняющих веществ в атмосферном воздухе Москвы и ряда крупных городов мира (Лондон, Париж, Гонконг, Стокгольм, Нью-Йорк и др.) показывает, что в целом уровень загрязнения воздушного бассейна в городе Москве сравним с уровнем загрязнения в других городах.

Среднегодовые концентрации загрязняющих веществ во всех рассматриваемых городах характеризуются высокой пространственной изменчивостью. Максимальный уровень загрязнения воздуха как в Москве, так и в других крупных городах, отмечается на территориях, прилегающих к крупным автотрассам.

Подробная информация о загрязнении воздуха в Москве представлена  на сайте ГПБУ «Мосэкомониторинг» (www.mosecom.ru). На сайте размещается информация по всем действующим АСКЗА на территории Москвы в режиме реального времени за текущий день, неделю, месяц и год, а также публикуются квартальные бюллетени, ежегодные госдоклады.

Материал подготовлен на основе информации: http://data.mos.ru/, http://www.mosecom.ru,  ГН 2.1.6.1338-03 «Предельно допустимые концентрации (ПДК) загрязняющих веществ в атмосферном воздухе населенных мест», РД 52.04.186-89 «Руководство по контролю загрязнения атмосферы», Федерального закона  № 96-ФЗ «Об охране атмосферного воздуха».

In [28]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=215804")
csvname = unzip(filename)

In [29]:
ecology_mean = pd.read_csv(csvname, sep=';')
ecology_mean = ecology_mean.drop(index=0)
ecology_mean = ecology_mean.drop(columns=[col for col in ecology_mean.columns if col.startswith("Unnamed:")])
ecology_mean.head()

,ID,Period,global_id,StationName,SurveillanceZoneCharacteristics,AdmArea,District,Location,Parameter,MonthlyAverage,MonthlyAveragePDKss
1,1,02.2016,272777248,Лосиный остров,Природные территории,Восточный административный округ,район Метрогородок,"улица Ротерта, дом 4, строение 5",Оксид углерода,0.19,0.06
2,2,02.2016,272777249,Глебовская,Смешанные антропогенные источники загрязнений,Восточный административный округ,район Богородское,"Глебовская улица, дом 3, корпус 1",Оксид углерода,0.32,0.11
3,3,02.2016,272777250,Глебовская,Смешанные антропогенные источники загрязнений,Восточный административный округ,район Богородское,"Глебовская улица, дом 3, корпус 1",Сероводород,0.001,NaN
4,4,02.2016,272777251,Глебовская,Смешанные антропогенные источники загрязнений,Восточный административный округ,район Богородское,"Глебовская улица, дом 3, корпус 1",Оксид азота,0.014,0.23
5,5,02.2016,272777252,Глебовская,Смешанные антропогенные источники загрязнений,Восточный административный округ,район Богородское,"Глебовская улица, дом 3, корпус 1",Диоксид азота,0.024,0.59


In [30]:
os.remove(filename)
os.remove(csvname)

In [31]:
ecology_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30381 entries, 1 to 30381
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ID                               30381 non-null  object
 1   Period                           30381 non-null  object
 2   global_id                        30381 non-null  object
 3   StationName                      30381 non-null  object
 4   SurveillanceZoneCharacteristics  30381 non-null  object
 5   AdmArea                          30381 non-null  object
 6   District                         30381 non-null  object
 7   Location                         30381 non-null  object
 8   Parameter                        30381 non-null  object
 9   MonthlyAverage                   30381 non-null  object
 10  MonthlyAveragePDKss              17761 non-null  object
dtypes: object(11)
memory usage: 2.5+ MB


## Датасет землянных работ

[Страница](https://data.mos.ru/opendata/62501)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218818)

В соответствии с Правилами проведения земляных работ, установки временных ограждений, размещения временных объектов в городе Москве, утвержденных Постановлением Правительства Москвы от 19.05.2015 № 299-ПП.

Данные о действующих зарегистрированных уведомлениях на проведения земляных работ (установки временных ограждений, размещения временных объектов), осуществляемых в целях проведения работ, финансируемых за счет средств бюджета города Москвы или за счет средств Московского фонда реновации жилой застройки, автономной некоммерческой организации "Развитие социальной инфраструктуры", автономной некоммерческой организации "Развитие спортивных и инфраструктурных объектов" включают в себя:

    Номер уведомления
    Дата регистрации
    Наименование организации-балансодержателя коммуникаций
    Наименование организации-производителя работ
    Цель производства работ
    Назначения объекта
    Адрес близлежащего строения
    Округ
    Район
    Плановая дата начала работ
    Плановая дата окончания работ

In [32]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=218818")
csvname = unzip(filename)

In [33]:
ecology_work = pd.read_csv(csvname, sep=';')
ecology_work = ecology_work.drop(index=0)
ecology_work = ecology_work.drop(columns=[col for col in ecology_work.columns if col.startswith("Unnamed:")])
ecology_work.head()

,RegistrationNumberNotifications,Date,StateCustomer,StateCustomerINN,WorkType,EarthworkObjectives,ObjectivesOfTheInstallationOfTemporaryFences,ObjectivesOfThePlacementOfTemporaryObjects,AdmArea,District,AddressOfNearbyBuilding,WorkPlaceDescription,WorkStartDate,WorkEndDate,Executor,ExecutorINN,global_id,geoData,geodata_center
1,У0208103,19.02.2021,Московский фонд реновации жилой застройки,7703434808,1. Земляные работы,3. Инженерно-геологические изыскания,NaN,NaN,Северо-Восточный административный округ,Бутырский район,NaN,"Яблочкова ул., вл. 26-28,",22.02.2021,30.09.2024,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""МОНОЛИТНОЕ СТРОИТЕЛЬНОЕ УПРАВЛЕНИЕ-1""",7745000111,1092198818,"{coordinates=[[37.58037861, 55.825088698], [37.580234128, 55.824253023], [37.579996429, 55.824365685], [37.58015831, 55.82440844], [37.580110955, 55.824520978], [37.579997166, 55.824724637], [37.579883299, 55.824890793], [37.581054021, 55.824932887], [37.580624454, 55.8243117], [37.580910526, 55.824579387], [37.580834711, 55.824734804], [37.580720825, 55.824890246], [37.580606916, 55.825034973], [37.580558908, 55.82483142], [37.580520562, 55.824697508]], type=MultiPoint}",NaN
2,У0708913,19.08.2021,Московский фонд реновации жилой застройки,7703434808,"[2. Установка временных ограждений, 3. Размещение временных объектов]",NaN,"1. Строительство, реконструкция объектов капитального строительства","1. Строительство, реконструкция объектов капитального строительства",Западный административный округ,район Проспект Вернадского,NaN,"Лобачевского ул., вл. 28-36,",23.08.2021,23.08.2024,"Общество с ограниченной ответственностью ""Магистраль""",6325045051,1139037822,"{coordinates=[[[37.503629953, 55.670195783], [37.506739318, 55.670206366], [37.506758129, 55.669349144], [37.503610916, 55.669359994], [37.503629953, 55.670195783]]], type=Polygon}",NaN
3,У0306953/1,09.09.2021,Государственное казенное учреждение города Москвы «Управление дорожно-мостового строительства»,7728381587,"[1. Земляные работы, 2. Установка временных ограждений, 3. Размещение временных объектов]","[1. Строительство, реконструкция объектов капитального строительства, 2. Прокладка инженерных сетей и сооружений к строящимся (реконструируемым) объектам капитального строительства]","[1. Строительство, реконструкция объектов капитального строительства, 2. Прокладка инженерных сетей и сооружений к строящимся (реконструируемым) объектам капитального строительства]","[1. Строительство, реконструкция объектов капитального строительства, 2. Прокладка инженерных сетей и сооружений к строящимся (реконструируемым) объектам капитального строительства]",Восточный административный округ,район Косино-Ухтомский,NaN,"Москва, ВАО, Косино - Ухтомский, районы Косино - Ухтомский, Некрасовка, улица Дмитриевского, 16А, улица Дмитриевского, 11, Красковская улица, 4, Сочинская улица 6к2",09.09.2021,02.09.2024,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""МОНОЛИТНОЕ СТРОИТЕЛЬНОЕ УПРАВЛЕНИЕ-1""",7745000111,1139362004,"{coordinates=[[[37.915712894, 55.709283775], [37.915802417, 55.709448219], [37.915942536, 55.709462478], [37.915997848, 55.709309598], [37.915712894, 55.709283775]]], type=Polygon}",NaN
4,У0308980,05.10.2021,Московский фонд реновации жилой застройки,7703434808,"[1. Земляные работы, 2. Установка временных ограждений, 3. Размещение временных объектов]","[1. Строительство, реконструкция объектов капитального строительства, 9. Снос зданий и сооружений, ликвидация коммуникаций (за исключением сноса в целях строительства, реконструкции объекта капитального строительства)]","[1. Строительство, реконструкция объектов капитального строительства, 9. Снос зданий и сооружений, ликвидация коммуникаций (за исключением сноса в целях строительства, реконструкции объекта капитального строительства)]","[1. Строительство, реконструкция объектов капитального строительства, 9. Снос зданий и сооружений, ликвидация коммуникаций (за исключением сноса в целях строительства, реконструкции объекта капитального строительства)]",Восточный административный округ,район Гольяново,NaN,"г

In [34]:
os.remove(filename)
os.remove(csvname)

In [35]:
ecology_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6641 entries, 1 to 6641
Data columns (total 19 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   RegistrationNumberNotifications               6641 non-null   object
 1   Date                                          6641 non-null   object
 2   StateCustomer                                 6641 non-null   object
 3   StateCustomerINN                              6641 non-null   object
 4   WorkType                                      6641 non-null   object
 5   EarthworkObjectives                           992 non-null    object
 6   ObjectivesOfTheInstallationOfTemporaryFences  6585 non-null   object
 7   ObjectivesOfThePlacementOfTemporaryObjects    2274 non-null   object
 8   AdmArea                                       6641 non-null   object
 9   District                                      6641 non-null   object
 10  

## Датасет по капремонту

[Страница](https://data.mos.ru/opendata/62963)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218822)

Каталог набора открытых данных «Работы по капитальному ремонту многоквартирных домов» позволяет получить информацию о ходе выполнения работ на объектах городского хозяйства в части мероприятий развития государственных программ города Москвы и обеспечения своевременного и качественного сбора данных по работам, выполняемым на объектах городского хозяйства.

In [36]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=218822")
csvname = unzip(filename)

In [37]:
ecology_capital = pd.read_csv(csvname, sep=';')
ecology_capital = ecology_capital.drop(index=0)
ecology_capital = ecology_capital.drop(columns=[col for col in ecology_capital.columns if col.startswith("Unnamed:")])
ecology_capital.head()

,global_id,ObjectAddress,AdmArea,District,WorkEssence,DetailedWork,WorkName,YearOfWork,ContractType,ContractNumber,...,CustomerID,ContractorName,ContractStartDate,ContractEndDate,ContractStatus,StartDate,StartDateActual,EndDate,EndDateActual,WorkStatus
1,1136594363,nested data,Северный административный округ,Савёловский район,Цоколь,Ремонт,Ремонт Цоколь,2015,государственный контракт,2770584037915000188,...,6777,Общество с ограниченной ответственностью «МАНСТАР II»,10.08.2015,31.12.2015,Исполнен,10.08.2015,10.08.2015,31.10.2015,31.10.2015,завершена
2,1136594367,nested data,Зеленоградский административный округ,район Матушкино,Пандус для МГН,Установка,Установка Пандус для МГН,2017,государственный контракт,2773556630017000027,...,3152,Капранова мария александровна,16.06.2017,30.09.2017,Исполнен,01.08.2017,01.08.2017,30.09.2017,02.08.2017,завершена
3,1136594369,nested data,Северный административный округ,район Беговой,Электрическое вводно-распределительное устройство,Замена,Замена Электрическое вводно-распределительное устройство,2015,государственный контракт,2770584037915000160,...,6777,Общество с ограниченной ответственностью «МАНСТАР II»,22.07.2015,31.12.2015,Исполнен,21.08.2015,21.08.2015,20.09.2015,20.09.2015,завершена
4,1136594373,nested data,Северо-Восточный административный округ,район Бибирево,Оконно-балконные блоки,Замена,Замена Оконно-балконные блоки,2019,государственный контракт,2771548881519000019,...,4692678,Морозков вячеслав борисович,08.04.2019,31.12.2019,Исполнен,15.04.2019,15.04.2019,31.08.2019,31.07.2019,завершена
5,1136594376,nested data,Восточный административный округ,район Новокосино,Пандус для МГН,Обустройство,Обустройство Пандус для МГН,2017,государственный контракт,2772027068417000007,...,2825,ООО «СтройКом 2003»,30.05.2017,31.12.2017,Исполнен,01.02.2017,30.05.2017,01.10.2017,27.10.2017,завершена


In [38]:
os.remove(filename)
os.remove(csvname)

In [39]:
ecology_capital.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3490 entries, 1 to 3490
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   global_id          3490 non-null   object
 1   ObjectAddress      3490 non-null   object
 2   AdmArea            3490 non-null   object
 3   District           3490 non-null   object
 4   WorkEssence        3490 non-null   object
 5   DetailedWork       3490 non-null   object
 6   WorkName           3490 non-null   object
 7   YearOfWork         3490 non-null   object
 8   ContractType       3490 non-null   object
 9   ContractNumber     3490 non-null   object
 10  CustomerName       3490 non-null   object
 11  CustomerID         3490 non-null   object
 12  ContractorName     3490 non-null   object
 13  ContractStartDate  3490 non-null   object
 14  ContractEndDate    3490 non-null   object
 15  ContractStatus     3490 non-null   object
 16  StartDate          3490 non-null   object


## Датасет ремонтных работ ЖКХ

[Страница](https://data.mos.ru/opendata/62441)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=216816)

Набор данных содержит информацию о локальных ремонтных работах на объектах городского хозяйства Москвы, которые проводятся учреждениями подведомственными Департаменту жилищно-коммунального хозяйства города Москвы.За исключением работ, проводимых: в рамках уведомлений Объединения административно-технических инспекций города Москвы (ОАТИ) на аварийно-восстановительный ремонт инженерных коммуникаций, сооружений и дорог; в рамках уведомлений ОАТИ на проведение земляных работ, осуществляемых в целях проведения работ, финансируемых за счет средств бюджета города Москвы; в рамках ордеров ОАТИ на производство земляных работ, установку временных ограждений и размещение временных объектов в пределах установленных границ города Москвы.

Работы по локальному ремонту асфальтобетонного покрытия на улично-дорожной сети города Москвы проводятся в рамках технического содержание объектов дорожного хозяйства (далее также - ОДХ).

- техническое содержание ОДХ - комплекс работ по устранению (ремонту) возникающих локальных повреждений и дефектов дорожных покрытий,.

Материал подготовлен на основании постановления Правительства Москвы от 16 декабря 2014 г. N 762-ПП «Об утверждении требований к санитарно-техническому содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы и порядка выполнения работ по капитальному ремонту, текущему ремонту, разметке и содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы

In [40]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=216816")
csvname = unzip(filename)

In [41]:
ecology_repair = pd.read_csv(csvname, sep=';')
ecology_repair = ecology_repair.drop(index=0)
ecology_repair = ecology_repair.drop(columns=[col for col in ecology_repair.columns if col.startswith("Unnamed:")])
ecology_repair.head()

,global_id,Customer,WorkReason,ActualStartDate,ActualEndDate,AdmArea,District,AddressOfNearbyBuilding,WorkPlaceDescription,geoData,geodata_center
1,2375907599,ГУП Мосводосток,Заявка на портал «Наш город»,09.07.2022,09.07.2022,Северо-Восточный административный округ,район Северный,NaN,Дмитровское шоссе д.167А,"{coordinates=[[37.543292344, 55.91761981], [37.543714328, 55.919591187]], type=MultiPoint}","{coordinates=[37.543292344, 55.91761981], type=Point}"
2,2375907618,ГУП Мосводосток,Плановая работа,09.07.2022,09.07.2022,Северо-Восточный административный округ,Алтуфьевский район,NaN,Бибиревская ул. д.2 с.4,"{coordinates=[[37.58924489, 55.879439461], [37.589930999, 55.879396102]], type=MultiPoint}","{coordinates=[37.589930999, 55.879396102], type=Point}"
3,2375907683,ГУП Мосводосток,Плановая работа,10.07.2022,10.07.2022,Северо-Восточный административный округ,район Отрадное,NaN,Алтуфьевское шоссе д.28 к1 дублер,"{coordinates=[[37.585029845, 55.865231807], [37.58506923, 55.865815743]], type=MultiPoint}","{coordinates=[37.585029845, 55.865231807], type=Point}"
4,2375907690,ГУП Мосводосток,Плановая работа,10.07.2022,10.07.2022,Северо-Восточный административный округ,Алтуфьевский район,NaN,Бибиревская ул. д.1,"{coordinates=[[37.588682757, 55.879517551]], type=MultiPoint}","{coordinates=[37.588682757, 55.879517551], type=Point}"
5,2375907692,ГУП Мосводосток,Плановая работа,10.07.2022,10.07.2022,Северо-Восточный административный округ,Алтуфьевский район,NaN,Бибиревская ул д.12А .,"{coordinates=[[37.602415264, 55.882662421]], type=MultiPoint}","{coordinates=[37.602415264, 55.882662421], type=Point}"


In [42]:
os.remove(filename)
os.remove(csvname)

In [43]:
ecology_repair.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31188 entries, 1 to 31188
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   global_id                31188 non-null  object
 1   Customer                 31188 non-null  object
 2   WorkReason               31188 non-null  object
 3   ActualStartDate          31188 non-null  object
 4   ActualEndDate            31188 non-null  object
 5   AdmArea                  31188 non-null  object
 6   District                 31188 non-null  object
 7   AddressOfNearbyBuilding  1 non-null      object
 8   WorkPlaceDescription     31188 non-null  object
 9   geoData                  31188 non-null  object
 10  geodata_center           31188 non-null  object
dtypes: object(11)
memory usage: 2.6+ MB


## Датасет ремонта поликлиник

[Страница](https://data.mos.ru/opendata/62581)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=208952)

Проведение мероприятий по капитальному ремонту объектов здравоохранения в городе Москве осуществляется в рамках Государственной программы города Москвы «Развитие здравоохранения города Москвы (Столичное здравоохранение)».

В план работ на 2021 год включены 44 объекта, площадью более 181 тыс.кв.м. Из них 26 взрослых и 16 детских поликлиник, а также 2 медицинских учреждения смешанного типа.

В рамках капитального ремонта, в соответствии с утвержденным стандартом оснащения поликлиник «Новый московский стандарт поликлиник» выполняется: замена всех инженерных систем, лифтов, окон и дверей, устройство вентиляции и кондиционирования, утепление фасадов и высококачественная отделка помещений, интеграция в системы «ЕМИАС», приспособление помещений для нужд маломобильных граждан, устройство системы доступа.

Кроме того, будут благоустроены территории, прилегающие к этим 44 поликлиникам 2021 года. Пространства около них обустроят в единой стилистике. Общая площадь благоустройства составит более 26 га.

In [44]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=208952")
csvname = unzip(filename)

In [45]:
ecology_clinic = pd.read_csv(csvname, sep=';')
ecology_clinic = ecology_clinic.drop(index=0)
ecology_clinic = ecology_clinic.drop(columns=[col for col in ecology_clinic.columns if col.startswith("Unnamed:")])
ecology_clinic.head()

,ObjectImpName,WorkType,WorkArea,Address,AdmArea,District,WorkYear,Work,ContractNumber,Customer,Contractor,PlannedStartDate,PlannedEndDate,ActualStartDate,ActualEndDate,WorkStatus,global_id,geoData,geodata_center
1,ГБУЗ «ГП № 220 ДЗМ» филиал №1,Работы по капитальному ремонту городской поликлиники,NaN,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Пресненский, Шмитовский проезд, дом 25, строение 1",Центральный административный округ,Пресненский район,2021,nested data,№ 2 - АНО,АНО «Развитие городских технологий»,ООО «Акпстройинвест»,25.02.2021,15.09.2021,25.02.2021,05.09.2022,Завершены,1132408905,"{coordinates=[37.542727925, 55.756209523], type=Point}",NaN
2,ГБУЗ «Детская городская поликлиника №150 Департамента здравоохранения города Москвы» Филиал №1,Работы по капитальному ремонту городской поликлиники,NaN,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Люблино, проспект 40 лет Октября, дом 25",Юго-Восточный административный округ,район Люблино,2021,nested data,32009870012,ГАУ МосжилНИИпроект,ООО Центр Инженерных Систем,18.01.2021,01.11.2021,18.01.2021,16.05.2022,Завершены,1132650469,"{coordinates=[37.74682819, 55.674092207], type=Point}",NaN
3,"ГБУЗ «Детская городская поликлиника № 143 Департамента здравоохранения города Москвы», Филиал №3",Работы по капитальному ремонту городской поликлиники,NaN,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Выхино-Жулебино, Ферганская улица, дом 10, корпус 2",Юго-Восточный административный округ,район Выхино-Жулебино,2021,nested data,32009860476,ГАУ МосжилНИИпроект,ООО ЮВИСТРОЙ,03.02.2021,15.07.2021,03.02.2021,18.07.2022,Завершены,1132654688,"{coordinates=[37.807924004, 55.708436631], type=Point}",NaN
4,"ГБУЗ Детская городская поликлиника № 7 Департамента здравоохранения города Москвы, Филиал № 4",Работы по капитальному ремонту городской поликлиники,NaN,"город Москва, 2-я Владимирская улица, дом 29А",Восточный административный округ,район Перово,2021,nested data,32009869989,ГАУ МосжилНИИпроект,ООО Эксперт-Строй,18.01.2021,15.07.2021,18.01.2021,18.07.2022,Завершены,1132656958,"{coordinates=[37.783489883, 55.755482624], type=Point}",NaN
5,"ГБУЗ «Детская городская поликлиника № 148 Департамента здравоохранения города Москвы», филиал № 3",Работы по капитальному ремонту городской поликлиники,NaN,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Марьино, Новочеркасский бульвар, дом 35",Юго-Восточный административный округ,район Марьино,2021,nested data,32110028847,ГАУ МосжилНИИпроект,АО ГРУППА КОМПАНИЙ ЕКС,18.01.2021,01.11.2021,18.01.2021,18.04.2022,Завершены,1132658258,"{coordinates=[37.73405259, 55.649738673], type=Point}",NaN


In [46]:
os.remove(filename)
os.remove(csvname)

In [47]:
ecology_clinic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 1 to 263
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ObjectImpName     263 non-null    object
 1   WorkType          263 non-null    object
 2   WorkArea          0 non-null      object
 3   Address           263 non-null    object
 4   AdmArea           263 non-null    object
 5   District          263 non-null    object
 6   WorkYear          263 non-null    object
 7   Work              263 non-null    object
 8   ContractNumber    263 non-null    object
 9   Customer          263 non-null    object
 10  Contractor        263 non-null    object
 11  PlannedStartDate  263 non-null    object
 12  PlannedEndDate    263 non-null    object
 13  ActualStartDate   41 non-null     object
 14  ActualEndDate     48 non-null     object
 15  WorkStatus        263 non-null    object
 16  global_id         263 non-null    object
 17  geoData         

## Датасет дорожных работ

[Страница](https://data.mos.ru/opendata/62101)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=192235)

Объект дорожного хозяйства - искусственное сооружение, предназначенное для безопасного движения транспорта и пешеходов в любое время года независимо от природно-климатических условий.

К объектам дорожного хозяйства относятся городские магистрали, улицы, проезды, площади, переулки и другие объекты, определенные Классификацией объектов дорожного хозяйства, утвержденной постановлением Правительства Москвы от 9 ноября 1999 г. N 1018 "Об утверждении Правил санитарного содержания территорий, организации уборки и обеспечения чистоты и порядка в г. Москве".

Текущий ремонт ОДХ - это комплекс работ по восстановлению транспортно-эксплуатационного состояния отдельных элементов ОДХ до уровня, отвечающего нормам безопасного движения автотранспорта и пешеходов.

Задача текущего ремонта состоит в восстановлении транспортно-эксплуатационного состояния отдельных элементов ОДХ до уровня, позволяющего обеспечить выполнение нормативных требований в период до очередного капитального или текущего ремонта при интенсивности движения, не превышающей расчетную для данного ОДХ.

Критерием для назначения текущего ремонта элементов ОДХ является снижение потребительских свойств ОДХ до предельно допустимых значений, когда их восстановление работами по техническому содержанию невозможно или экономически нецелесообразно.

Текущий ремонт дорожного покрытия картами размером от 100 кв. м и выше:

- восстановление изношенного верхнего слоя дорожной одежды с обеспечением требуемой ровности и шероховатости;

- ликвидация колей и неровностей на проезжей части с заменой нестабильных слоев дорожного покрытия с обязательным фрезерованием на ширину одной или нескольких полос движения или на всю ширину покрытия с укладкой нового слоя из сдвигоустойчивого и износостойкого асфальтобетона;

- при необходимости устройство покрытий из асфальтобетонных смесей на тротуарах с предварительным фрезерованием или разборкой.

Материал подготовлен на основании постановления Правительства Москвы от 16 декабря 2014 г. N 762-ПП «Об утверждении требований к санитарно-техническому содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы и порядка выполнения работ по капитальному ремонту, текущему ремонту, разметке и содержанию объектов дорожного хозяйства улично-дорожной сети города Москвы», а также открытых ресурсов сети Интернет.


In [48]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=192235")
csvname = unzip(filename)

In [49]:
ecology_road = pd.read_csv(csvname, sep=';')
ecology_road = ecology_road.drop(index=0)
ecology_road = ecology_road.drop(columns=[col for col in ecology_road.columns if col.startswith("Unnamed:")])
ecology_road.head()

,WorksPlace,WorkYear,OnTerritoryOfMoscow,AdmArea,District,WorksBeginDate,PlannedEndDate,ActualBeginDate,ActualEndDate,WorksType,WorksStatus,WorkReason,Customer,Contractor,global_id,geoData,geodata_center
1,2-й Звенигородский переулок,2020,да,Центральный административный округ,Пресненский район,14.08.2020,08.10.2020,14.08.2020,08.10.2020,текущий ремонт асфальтобетонного покрытия,закончены,"Титульный список объектов улично-дорожной сети города Москвы, подлежащих ремонту",ГБУ АВД,NaN,1031138031,"{coordinates=[[[37.555009201, 55.764145825], [37.55523361, 55.764130143], [37.555469645, 55.764852984], [37.555888069, 55.765439715], [37.556891215, 55.766622223], [37.556703461, 55.766740169], [37.556193841, 55.766050631], [37.555839789, 55.765630249], [37.555249703, 55.764834837], [37.555008305, 55.76414829]]], type=MultiLineString}","{coordinates=[37.555888069, 55.765439715], type=Point}"
2,2-я Звенигородская улица,2020,да,Центральный административный округ,Пресненский район,28.08.2020,13.09.2020,28.08.2020,13.09.2020,текущий ремонт асфальтобетонного покрытия,закончены,"Титульный список объектов улично-дорожной сети города Москвы, подлежащих ремонту",ГБУ АВД,NaN,1031138034,"{coordinates=[[[37.555076618, 55.75643665], [37.555360933, 55.756475975], [37.555162449, 55.75778278], [37.555044432, 55.758992745], [37.555010206, 55.760671568], [37.555046082, 55.76205522], [37.555024188, 55.762303149], [37.555088561, 55.763921281], [37.554895442, 55.763969672], [37.554841798, 55.763664199], [37.554852527, 55.763171202], [37.554825705, 55.761005571], [37.554847163, 55.759759371], [37.554852527, 55.758797471], [37.554986637, 55.757620774], [37.555142206, 55.756447067]]], type=MultiLineString}","{coordinates=[37.554847163, 55.759759371], type=Point}"
3,2-я Черногрязская улица,2020,да,Центральный административный округ,Пресненский район,14.08.2020,02.10.2020,14.08.2020,02.10.2020,текущий ремонт асфальтобетонного покрытия,закончены,"Титульный список объектов улично-дорожной сети города Москвы, подлежащих ремонту",ГБУ АВД,NaN,1031138035,"{coordinates=[[[37.547976298, 55.76482853], [37.548201604, 55.764801311], [37.547960205, 55.764368817], [37.547740264, 55.763881878], [37.547257466, 55.762548058], [37.546701192, 55.761068673], [37.546432971, 55.760403233], [37.546289349, 55.760426327], [37.546412731, 55.760722752], [37.546635354, 55.7613897], [37.546737278, 55.761693679], [37.54713471, 55.762741806], [37.547630919, 55.763968252], [37.547942055, 55.76478334], [37.547969073, 55.764824498]]], type=MultiLineString}","{coordinates=[37.547257466, 55.762548058], type=Point}"
4,Бахрушина ул.,2020,да,Центральный административный округ,район Замоскворечье,21.03.2020,03.06.2020,21.03.2020,03.06.2020,текущий ремонт асфальтобетонного покрытия,закончены,"Титульный список объектов улично-дорожной сети города Москвы, подлежащих ремонту",ГБУ Авд,NaN,1031138040,"{coordinates=[[[37.637720128, 55.731480493], [37.637988349, 55.731583411], [37.637473923, 55.732245021], [37.637028118, 55.732903161], [37.636223455, 55.734129035], [37.634887376, 55.73607888], [37.634418723, 55.736813639], [37.634023845, 55.737432723], [37.633857548, 55.737363112], [37.634136498, 55.736903074], [37.634699762, 55.736140367], [37.635075271, 55.735604647], [37.635806609, 55.734396363], [37.636316229, 55.7336215], [37.637741268, 55.731507124]]], type=MultiLineString}","{coordinates=[37.635806609, 55.734396363], type=Point}"
5,Библиотечная ул. (от ул. Б.Андроньевская до ул. Добровольческая),2020,да,Центральный административный округ,Таганский район,08.03.2020,20.03.2020,08.03.2020,20.03.2020,текущий ремонт асфальтобетонного покрытия,закончены,"Титульный список объектов улично-дорожной сети города Москвы, подлежащих ремонту",гбу авд,NaN,1031138042,"{coordinates=[[[37.668564862, 55.744265971], [37.668583637, 55.744228146], [37.670557743, 55.744371881], [37.670530921, 55.744458122], [37.66860947, 55.744302349], [37.668556815, 55.74430077]]], type=MultiLineString}","{coordinates=[37.66860947, 55.744302

In [50]:
os.remove(filename)
os.remove(csvname)

In [51]:
ecology_road.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 1 to 2200
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   WorksPlace           2200 non-null   object
 1   WorkYear             2200 non-null   object
 2   OnTerritoryOfMoscow  2200 non-null   object
 3   AdmArea              2184 non-null   object
 4   District             2184 non-null   object
 5   WorksBeginDate       2200 non-null   object
 6   PlannedEndDate       2200 non-null   object
 7   ActualBeginDate      2004 non-null   object
 8   ActualEndDate        2004 non-null   object
 9   WorksType            2200 non-null   object
 10  WorksStatus          2200 non-null   object
 11  WorkReason           2200 non-null   object
 12  Customer             2200 non-null   object
 13  Contractor           24 non-null     object
 14  global_id            2200 non-null   object
 15  geoData              2200 non-null   object
 16  geodat

## Датасет благоустройства парков

[Страница](https://data.mos.ru/opendata/61041)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=208745)

Проведение мероприятий по благоустройству парков и озелененных территорий в Москве осуществляется в рамках Государственной программы города Москвы «Развитие городской среды», утвержденной постановлением Правительства Москвы № 476-ПП от 7 октября 2011 года (ред. от 10.10.2016 г.).

Благоустройство парковых территорий носит комплексный характер. Как правило, при благоустройстве парковых территорий обеспечивается проведение следующих видов работ:

- озеленение (посадка деревьев и кустарников, разбивка газонов и цветников, создание элементов вертикального и мобильного озеленения);

- обустройство пляжных зон и пристаней;

- создание искусственных водоёмов и фонтанов;

- создание дорожно-тропиночной сети;

- установка малых архитектурных форм (элементы монументально-декоративного оформления, трельяжи, шпалеры, перголы, цветочницы, вазоны, городская мебель: скамейки, столы, садовые диваны, урны, контейнеры для сбора мусора);

- организация зон тихого и активного отдыха, детских и спортивных площадок;

- повсеместное современное функциональное освещение территории - устройство навигационной и информационной инфраструктуры;

- обеспечение территории системами безопасности и видеонаблюдения;

- создание безбарьерной среды для маломобильных групп населения;

- иные виды работ, отвечающие тематике парковой территории.

В зависимости от функционального назначения парковой территории, ее пространственно-площадных характеристик выполняются те или иные виды указанных выше благоустроительных работ.


In [52]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=208745")
csvname = unzip(filename)

In [53]:
ecology_park = pd.read_csv(csvname, sep=';')
ecology_park = ecology_park.drop(index=0)
ecology_park = ecology_park.drop(columns=[col for col in ecology_park.columns if col.startswith("Unnamed:")])
ecology_park.head()

,global_id,ID,WorkType,AdmArea,District,ObjectCitySignificance,Works,WorkScopeModifyReason,ContractNumber,ContractorName,...,WorkYear,StartDatePlanned,EndDatePlanned,StartDateActual,EndDateActual,WorkEntityState,WorkDeleteReason,WorkStopReason,geoData,geodata_center
1,886230006,1,Создание и благоустройство парков и озелененных территорий,Юго-Восточный административный округ,район Печатники,Парк технических видов спорта (район Печатники),nested data,NaN,153-ГКУДКР-19,ООО СТРОЙСТАНДАРТ,...,2019,02.07.2018,31.12.2019,02.07.2018,31.12.2019,Завершены,NaN,NaN,"{coordinates=[[[37.688792534, 55.651629655], [37.684838958, 55.653333531], [37.682167478, 55.651888237], [37.679356523, 55.650210839], [37.674285807, 55.652873068], [37.673034556, 55.654006493], [37.672806568, 55.654380692], [37.672736831, 55.654692332], [37.672683187, 55.654975915], [37.672661729, 55.655470285], [37.681802697, 55.655094201], [37.689055391, 55.651818473], [37.688792534, 55.651629655]]], type=Polygon}",NaN
2,886243735,9,Создание и благоустройство парков и озелененных территорий,Западный административный округ,Можайский район,Центральный парк ИЦ «Сколково»,nested data,NaN,0173200001418000067,ООО ЭКО ГРИН СТРОЙ,...,2018,02.04.2018,31.12.2018,02.04.2018,31.12.2018,Завершены,NaN,NaN,"{coordinates=[[[37.352357069, 55.693036916], [37.351949374, 55.695909251], [37.34903113, 55.695994084], [37.350468794, 55.696806054], [37.352142493, 55.696806054], [37.352131339, 55.697898684], [37.35470626, 55.697898684], [37.354727717, 55.696432319], [37.355264159, 55.6964202], [37.355285617, 55.696250534], [37.357710334, 55.696262653], [37.357581588, 55.695026496], [37.357367011, 55.694335685], [37.357259723, 55.693475184], [37.354534598, 55.693378225], [37.35386941, 55.692905546], [37.352357069, 55.693036916]]], type=Polygon}",NaN
3,886275796,10,Создание и благоустройство парков и озелененных территорий,Северный административный округ,Хорошёвский район,парк Ходынское поле,nested data,NaN,"0173200001418000520, 0173200001418000521",АО МОСКАПСТРОЙИНЖИНИРИНГ,...,2018,10.06.2018,31.12.2018,10.06.2018,31.12.2018,Завершены,NaN,NaN,"{coordinates=[[[37.522228657, 55.789157166], [37.522353382, 55.789231102], [37.528432202, 55.78847295], [37.538045481, 55.787248377], [37.537004784, 55.786553158], [37.535985545, 55.786087657], [37.534698085, 55.785543557], [37.533367709, 55.785205003], [37.532403614, 55.785008657], [37.531373645, 55.784924018], [37.529957439, 55.784899835], [37.528305198, 55.785044932], [37.526867534, 55.785371396], [37.525687362, 55.785770405], [37.52524748, 55.785981999], [37.524871971, 55.786157319], [37.524163867, 55.786586547], [37.523348476, 55.787221312], [37.522833492, 55.787765388], [37.522554542, 55.788321547], [37.522243406, 55.788980464], [37.522228657, 55.789157166]]], type=Polygon}",NaN
4,886302901,2,Создание и благоустройство парков и озелененных территорий,Зеленоградский административный округ,район Савёлки,Парк Победы,nested data,NaN,138-ГКУДКР-19,ООО АРТСТРОЙТЕХНОЛОГИЯ,...,2019,16.08.2018,01.07.2020,16.08.2018,30.08.2019,Завершены,NaN,NaN,"{coordinates=[[[37.223031307, 55.989731377], [37.222643251, 55.98942443], [37.222238861, 55.989378418], [37.222134878, 55.989260241], [37.221986101, 55.988962899], [37.224041895, 55.988742061], [37.223704436, 55.988504865], [37.222645151, 55.988578713], [37.222566712, 55.98842827], [37.222204969, 55.988285415], [37.221006376, 55.988395146], [37.220330718, 55.986484144], [37.225922503, 55.985679794], [37.2262357, 55.985356583], [37.224259332, 55.984374527], [37.223824814, 55.984215141], [37.223454669, 55.984127929], [37.2229826, 55.984109885], [37.218790769, 55.984715405], [37.217248714, 55.985415118], [37.216774318, 55.985544743], [37.216280792, 55.985586844], [37.215224001, 55.985514671], [37.214699953, 55.985518241], [37.214243754, 55.985580297], [37.212755974, 55.985992819], [37.212514575, 55.986143176], [37.213648713, 55.990557188], [37.213930854, 55.990953817], [37.214769483, 55.991657228], [37.21489877, 55.991673625],

In [54]:
os.remove(filename)
os.remove(csvname)

In [55]:
ecology_park.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 1 to 35
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   global_id               35 non-null     object
 1   ID                      35 non-null     object
 2   WorkType                35 non-null     object
 3   AdmArea                 35 non-null     object
 4   District                35 non-null     object
 5   ObjectCitySignificance  35 non-null     object
 6   Works                   35 non-null     object
 7   WorkScopeModifyReason   0 non-null      object
 8   ContractNumber          35 non-null     object
 9   ContractorName          35 non-null     object
 10  Customer                35 non-null     object
 11  WorkYear                35 non-null     object
 12  StartDatePlanned        35 non-null     object
 13  EndDatePlanned          35 non-null     object
 14  StartDateActual         35 non-null     object
 15  EndDateA

## Датасет благоустройства транспартной инфраструктуры

[Страница](https://data.mos.ru/opendata/62401)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215988)

Для обеспечения пешеходной и транспортной доступности, а также создания качественно новой городской среды, силами Департамента капитального ремонта города Москвы запланированы работы по улучшению пешеходной и транспортной доступности станций Московского метрополитена, станций Московских центральных диаметров, установке шумозащитных экранов, а также локальные мероприятия для обеспечения запуска наземного городского общественного транспорта.
С учетом моделирования транспортной ситуации на расчетный период, определившего существенный рост транспортных и пешеходных потоков, а также увеличения нагрузки на прилегающую к объектам улично-дорожную сеть, предусмотрены мероприятия по расширению и созданию пешеходных зон, приведению к нормативным показателям существующих тротуаров и проезжей части улиц, устройству освещения, оборудованию новых пешеходных переходов, светофорных объектов, установке новых и модернизации существующих остановок общественного транспорта, созданию парковочных мест, озеленению. Выполнение указанных работ позволит создать инфраструктуру, соответствующую современным требованиям как комфорта, так и безопасности.

In [56]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=215988")
csvname = unzip(filename)

In [57]:
ecology_trans = pd.read_csv(csvname, sep=';')
ecology_trans = ecology_trans.drop(index=0)
ecology_trans = ecology_trans.drop(columns=[col for col in ecology_trans.columns if col.startswith("Unnamed:")])
ecology_trans.head()

,WorkType,ObjectImpName,Area,AdmArea,District,Address,WorkPlaceDescription,WorkYear,Work,ContractNumber,Customer,Contractor,PlannedStartDate,PlannedEndDate,ActualStartDate,ActualEndDate,WorkStatus,global_id,geoData,geodata_center
1,"Благоустройство территорий, прилегающих к станциям метро",Станция метро Бутырская,NaN,Северо-Восточный административный округ,Бутырский район,NaN,Территория прилегающая к станции метро Бутырская,2021,nested data,2773161026921000055,ГКУ ДКР,ООО «АКП СТРОЙ-ИНВЕСТ»,14.05.2021,31.12.2021,14.05.2021,31.12.2021,Завершены,1133426192,"{coordinates=[[[37.606694698, 55.810915197], [37.602059841, 55.813869201], [37.602564096, 55.81405608], [37.602671385, 55.813977712], [37.602456808, 55.813881258], [37.604817152, 55.812259598], [37.605192661, 55.812126969], [37.605611086, 55.811849651], [37.606018782, 55.812048596], [37.606179714, 55.811946109], [37.605868578, 55.811753192], [37.606523037, 55.81129501], [37.607671022, 55.810505235], [37.607477903, 55.810420831], [37.606694698, 55.810915197]]], type=Polygon}",NaN
2,"Благоустройство территорий, прилегающих к станциям метро",Станция метро Мичуринский проспект,NaN,Западный административный округ,"[район Проспект Вернадского, район Раменки]",NaN,Территория прилегающая к станции метро Мичуринский проспект,2021,nested data,2773161026921000052,ГКУ ДКР,ООО «МИЛАНСТРОЙ»,06.05.2021,31.12.2021,06.05.2021,31.12.2021,Завершены,1133431319,"{coordinates=[[[37.492218018, 55.6818319], [37.491821051, 55.681662533], [37.490029335, 55.683223096], [37.489095926, 55.683307776], [37.489374876, 55.684075936], [37.488634586, 55.685013433], [37.4880445, 55.685261412], [37.488012314, 55.685751318], [37.487078905, 55.686144448], [37.487379313, 55.686634343], [37.486006022, 55.687475012], [37.486917973, 55.68733591], [37.487540245, 55.68657991], [37.487652898, 55.686623759], [37.486794591, 55.687794039], [37.487395406, 55.68781823], [37.487711906, 55.688159933], [37.488936782, 55.688874576], [37.48945713, 55.689548888], [37.491034269, 55.690561847], [37.492904663, 55.692298936], [37.494438887, 55.693411599], [37.49486804, 55.693816746], [37.49551177, 55.694629042], [37.495940924, 55.694362982], [37.494986057, 55.693631306], [37.494610548, 55.693286628], [37.493588626, 55.692353108], [37.492773235, 55.691635001], [37.491336465, 55.690206055], [37.48990953, 55.689447087], [37.489873767, 55.689218285], [37.487545609, 55.687636794], [37.487415075, 55.687456869], [37.489730716, 55.684460514], [37.490127683, 55.684124827], [37.490471005, 55.684103658], [37.490567565, 55.683891962], [37.492218018, 55.6818319]]], type=Polygon}",NaN
3,"Благоустройство территорий, прилегающих к станциям метро",Станция метро Новаторская,NaN,Юго-Западный административный округ,Обручевский район,NaN,Территория прилегающая к станции метро Новаторская,2021,nested data,2773161026921000047,ГКУ ДКР,ООО «ИНЖИНИРИНГ ГРУПП»,28.04.2021,31.12.2021,28.04.2021,31.12.2021,Завершены,1133433724,"{coordinates=[[[37.521476448, 55.670359446], [37.522906065, 55.669657574], [37.523793876, 55.669123599], [37.524243593, 55.66882106], [37.52476126, 55.668447421], [37.525216341, 55.668064198], [37.52566427, 55.667642144], [37.525849342, 55.667475742], [37.524516284, 55.666726922], [37.524194419, 55.666873662], [37.525388002, 55.667651221], [37.524783611, 55.66794318], [37.524067461, 55.667987049], [37.523775101, 55.667787368], [37.522712946, 55.668707103], [37.521558702, 55.66903082], [37.520939112, 55.668631468], [37.520636022, 55.668782738], [37.517154515, 55.666471262], [37.516915798, 55.666592285], [37.517256439, 55.666799536], [37.518152297, 55.667424308], [37.517648041, 55.667660297], [37.518359721, 55.668165047], [37.519046366, 55.667929061], [37.520524263, 55.668916864], [37.519862652, 55.66924663], [37.521476448, 55.670359446]]], type=Polygon}",NaN
4,"Благоустройство территорий, прилегающих к станциям метро",Станция метро Воронцовская,NaN,Юго-Западный административный округ,"[Обручевский район, район Черёмушки]",NaN,Территория приле

In [58]:
os.remove(filename)
os.remove(csvname)

In [59]:
ecology_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 1 to 107
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   WorkType              107 non-null    object
 1   ObjectImpName         107 non-null    object
 2   Area                  0 non-null      object
 3   AdmArea               107 non-null    object
 4   District              107 non-null    object
 5   Address               0 non-null      object
 6   WorkPlaceDescription  107 non-null    object
 7   WorkYear              107 non-null    object
 8   Work                  107 non-null    object
 9   ContractNumber        107 non-null    object
 10  Customer              107 non-null    object
 11  Contractor            107 non-null    object
 12  PlannedStartDate      107 non-null    object
 13  PlannedEndDate        107 non-null    object
 14  ActualStartDate       82 non-null     object
 15  ActualEndDate         61 non-null     ob

## Датасет по благоустройству городских пространств 


[Страница](https://data.mos.ru/opendata/63201)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218594)

In [60]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=218594")
csvname = unzip(filename)

In [61]:
ecology_city = pd.read_csv(csvname, sep=';')
ecology_city = ecology_city.drop(index=0)
ecology_city = ecology_city.drop(columns=[col for col in ecology_city.columns if col.startswith("Unnamed:")])
ecology_city.head()

,ObjectCategory,ObjectAddress,AdmArea,District,WorkName,WorkEssence,DetailedWork,YearOfWork,WorkVolume,ContractType,...,ContractStartDate,ContractEndDate,ContractStatus,StartDate,StartDateActual,EndDate,EndDateActual,WorkStatus,GovernmentProgram,global_id
1,"Скверы, парки, бульвары",nested data,Северо-Восточный административный округ,район Бибирево,Цветник Устройство,Цветник,Устройство,2022,nested data,государственный контракт,...,12.05.2022,31.12.2022,Исполнение,15.04.2022,15.04.2022,15.11.2022,15.11.2022,завершена,Туризм,2381786882
2,Нежилые здания,nested data,Центральный административный округ,Пресненский район,Фасад Ремонт,Фасад,Ремонт,2016,nested data,государственный контракт,...,27.04.2016,31.12.2017,Исполнен,27.04.2016,27.04.2016,31.12.2016,NaN,ведется,Туризм,2381803588
3,Нежилые здания,nested data,Центральный административный округ,Таганский район,Фасад Ремонт,Фасад,Ремонт,2016,nested data,государственный контракт,...,27.04.2016,31.12.2016,Исполнен,27.04.2016,27.04.2016,25.08.2016,25.08.2016,завершена,Туризм,2381803591
4,Нежилые здания,nested data,Центральный административный округ,район Якиманка,Фасад Ремонт,Фасад,Ремонт,2016,nested data,государственный контракт,...,26.04.2016,31.12.2016,Исполнен,26.04.2016,26.04.2016,24.08.2016,24.08.2016,завершена,Туризм,2381803593
5,Нежилые здания,nested data,Центральный административный округ,район Якиманка,Балконы Ремонт,Балконы,Ремонт,2016,nested data,государственный контракт,...,26.04.2016,31.12.2016,Исполнен,26.04.2016,26.04.2016,24.08.2016,24.08.2016,завершена,Туризм,2381803595


In [62]:
os.remove(filename)
os.remove(csvname)

In [63]:
ecology_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3084 entries, 1 to 3084
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ObjectCategory     3084 non-null   object
 1   ObjectAddress      3084 non-null   object
 2   AdmArea            3084 non-null   object
 3   District           3084 non-null   object
 4   WorkName           3084 non-null   object
 5   WorkEssence        3084 non-null   object
 6   DetailedWork       3084 non-null   object
 7   YearOfWork         3084 non-null   object
 8   WorkVolume         3084 non-null   object
 9   ContractType       3084 non-null   object
 10  ContractNumber     3084 non-null   object
 11  CustomerName       3084 non-null   object
 12  CustomerID         3084 non-null   object
 13  ContractorName     3084 non-null   object
 14  ContractStartDate  3084 non-null   object
 15  ContractEndDate    3084 non-null   object
 16  ContractStatus     3084 non-null   object


## Датасет по благоустройству жилой застройки 

[Страница](https://data.mos.ru/opendata/62961)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=218826)

In [64]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=218826")
csvname = unzip(filename)

In [65]:
ecology_livearea = pd.read_csv(csvname, sep=';')
ecology_livearea = ecology_livearea.drop(index=0)
ecology_livearea = ecology_livearea.drop(columns=[col for col in ecology_livearea.columns if col.startswith("Unnamed:")])
ecology_livearea.head()

/tmp/ipykernel_16811/3424006141.py:1: DtypeWarning: Columns (0,3,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  ecology_livearea = pd.read_csv(csvname, sep=';')


,global_id,AdmArea,District,YardID,WorkEssence,DetailedWork,WorkName,YearOfWork,ContractType,ContractNumber,...,CustomerID,ContractorName,ContractStartDate,ContractEndDate,ContractStatus,StartDate,StartDateActual,EndDate,EndDateActual,WorkStatus
1,1136473823,Северо-Восточный административный округ,Алексеевский район,138096,Бортовой (бордюрный) камень,Замена,Замена Бортовой (бордюрный) камень,2021,государственный контракт,2771779901221000009,...,4679637,"Общество с ограниченной ответственностью ""Дорсервис""",16.04.2021,31.12.2021,Исполнен,15.04.2021,16.04.2021,15.08.2021,10.08.2021,завершена
2,1136473824,Центральный административный округ,Красносельский район,138742,Бортовой (бордюрный) камень,Ремонт,Ремонт Бортовой (бордюрный) камень,2021,государственный контракт,2770880330421000015,...,5778,"Общество с ограниченной ответственностью ""Юго-Восточная строительная группа""",05.04.2021,31.12.2021,Исполнен,01.05.2021,01.05.2021,25.08.2021,28.08.2021,завершена
3,1136553437,Северный административный округ,Дмитровский район,2275321,Газон обыкновенный,Ремонт,Ремонт Газон обыкновенный,2019,государственный контракт,2771339866919000045,...,4711570,Бондаренко игорь михайлович,30.04.2019,15.08.2019,Исполнен,15.06.2019,15.06.2019,20.08.2019,25.06.2019,завершена
4,1136553438,Южный административный округ,район Бирюлёво Западное,3857,Цветочные конструкции (вертикальное озеленение),Устройство,Устройство Цветочные конструкции (вертикальное озеленение),2019,государственный контракт,2772494400619000016,...,4653791,"Общество с ограниченной ответственностью ""Дорсервис""",08.02.2019,30.09.2019,Исполнен,01.04.2019,01.04.2019,25.08.2019,28.05.2019,завершена
5,1136553441,Юго-Западный административный округ,Обручевский район,352960,Асфальтобетонное покрытие,Ремонт,Ремонт Асфальтобетонное покрытие,2018,государственный контракт,2772825275018000026,...,4705464,"Общество с ограниченной ответственностью ""Строительная компания альянс""",10.05.2018,31.12.2018,Исполнен,20.06.2018,21.06.2018,25.08.2018,02.07.2018,завершена


In [66]:
os.remove(filename)
os.remove(csvname)

In [67]:
ecology_livearea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127274 entries, 1 to 127274
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   global_id          127274 non-null  object
 1   AdmArea            127274 non-null  object
 2   District           127274 non-null  object
 3   YardID             127274 non-null  object
 4   WorkEssence        127274 non-null  object
 5   DetailedWork       127274 non-null  object
 6   WorkName           127274 non-null  object
 7   YearOfWork         127274 non-null  object
 8   ContractType       127274 non-null  object
 9   ContractNumber     127274 non-null  object
 10  CustomerName       127274 non-null  object
 11  CustomerID         127274 non-null  object
 12  ContractorName     127274 non-null  object
 13  ContractStartDate  127274 non-null  object
 14  ContractEndDate    127274 non-null  object
 15  ContractStatus     127274 non-null  object
 16  StartDate          1

## Датасет по благоустройству улиц

[Страница](https://data.mos.ru/opendata/62221)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=200723)

Проведение мероприятий по комлексному благоустройству улиц и общественных пространств в городе Москве осуществляется в рамках Государственной программы города Москвы «Развитие городской среды», утвержденной постановлением Правительства Москвы № 476-ПП от 7 октября 2011 года (ред. от 10.10.2016 г.).

Департаментом капитального ремонта города Москвы в рамках реализации данной программы в 2020-2023 годах запланировано проведение масштабных работ по комплексному благоустройству улиц, общественных пространств, природных и озеленённых территорий.

В план работ включены более 200 объектов, площадью более 4 800 га, среди них: благоустройство озеленённых территорий, природных комплексов и особо охраняемых природных территорий, благоустройство вылетных магистралей, улиц, общественных пространств и прочих объектов.

На всех реализуемых объектах запланированы мероприятия по комплексному благоустройству с созданием современных и технологичных общественных пространств, включающие в себя работы по: планировке территории, устройству покрытий дорожно-тропиночной сети, освещения и инженерных систем, установке малых архитектурных форм и оборудования, установке нестационарных объектов и прочие, направленных на социально – экономическое развитие города и создание комфортных условий для жителей.

In [68]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=200723")
csvname = unzip(filename)

In [69]:
ecology_street = pd.read_csv(csvname, sep=';')
ecology_street = ecology_street.drop(index=0)
ecology_street = ecology_street.drop(columns=[col for col in ecology_street.columns if col.startswith("Unnamed:")])
ecology_street.head()

WorkType   StreetName ObjectImpName  \
1  Работы по благоустройству улицы  nested data   nested data   
2  Работы по благоустройству улицы  nested data   nested data   
3  Работы по благоустройству улицы  nested data   nested data   
4  Работы по благоустройству улицы  nested data   nested data   
5  Работы по благоустройству улицы  nested data   nested data   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [70]:
os.remove(filename)
os.remove(csvname)

In [71]:
ecology_street.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 1 to 24
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   WorkType                 24 non-null     object
 1   StreetName               24 non-null     object
 2   ObjectImpName            24 non-null     object
 3   WorkAreaBounds           24 non-null     object
 4   UNOM                     0 non-null      object
 5   Address                  0 non-null      object
 6   Work                     24 non-null     object
 7   RegistryContractNumber   23 non-null     object
 8   EmergencyContractNumber  0 non-null      object
 9   ContractStatus           24 non-null     object
 10  Customer                 24 non-null     object
 11  Contractor               24 non-null     object
 12  WorkYear                 24 non-null     object
 13  PlannedStartDate         24 non-null     object
 14  PlannedEndDate           24 non-null     obj

## Датасет по помойкам

[Страница](https://data.mos.ru/opendata/2542)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=23148)

Каталог содержит подробную информацию о местах сбора отходов (контейнерные площадки, площадки для выкатных контейнеров), их адреса, район, административный округ, географические координаты, № площадки.
На территории Российской Федерации раздельный сбор отходов регламентируется положениями Федерального закона от 24.06.1998 № 89-ФЗ и законодательными актами субъектов РФ.
В городе Москве на  основании  постановлений  Правительства  Москвы  от  13.09.2012 № 485-ПП, от 02.10.2013 № 662-ПП и от 10.11.2014 № 644-ПП в Центральном, Северном, Северо-Восточном, Восточном, Юго-Восточном, Юго-Западном, Западном, Северо-Западном и Зеленоградском административных округах города Москвы реализуется новый комплексный метод обращения с отходами, для реализации которого на конкурсной основе заключено девять долгосрочных (на 15 лет) государственных контрактов на оказание услуг по обращению с твёрдыми бытовыми отходами и крупногабаритным мусором, образующимися в многоквартирных домах (далее - Контракт), расположенных на территориях указанных административных округов города Москвы.
Исполнителями Государственных контрактов являются ООО «МКМ-Логистика» (ЗАО, ЮЗАО), ООО «ЭкоЛайн» (ЦАО, САО), ООО «Хартия» (ВАО, СВАО), ООО «МСК-НТ» (ЮВАО, ЗелАО) и ООО «Спецтранс» (СЗАО).
По состоянию на 31.05.2017 в рамках Государственных контрактов в девяти округах создано 15307 мест сбора отходов (далее – МСО), в том числе 13771  для твердых бытовых отходов и 1536 для крупногабаритного мусора. В общей сложности на местах сбора отходов установлено 36687 контейнеров.
В ЦАО в рамках Государственных контрактов создано 1293 МСО, в том числе 1285  ТБО и 8 КГМ.
В САО в рамках Государственных контрактов создано 1806 МСО, в том числе 1627  ТБО, и 179 КГМ.
В СВАО в рамках Государственных контрактов создано 2481 МСО, в том числе 2075  ТБО, и 406 КГМ.
В ВАО в рамках Государственных контрактов создано 2759 МСО, в том числе 2465  ТБО, и 294 КГМ.
В ЮВАО в рамках Государственных контрактов  создано 2115 МСО, в том числе 1741  ТБО, и 374 КГМ.
В ЗелАО в рамках Государственных контрактов создано 383 МСО, в том числе 292  ТБО, и 91 КГМ.
В ЮЗАО в рамках Государственных контрактов  создано 2008 МСО, в том числе 1942  ТБО, и 66 КГМ.
В ЗАО в рамках Государственных контрактов создано 1489 МСО, в том числе 1405  ТБО, и 84 КГМ.
В СЗАО в рамках Государственных контрактов создано 973 МСО, в том числе 939  ТБО, и 34 КГМ.
А знаете ли Вы, что? Кроме того, по ряду МСО осуществляется сбор следующих вторичных материальных ресурсов: макулатуры, стеклянной тары, ПЭТФ-тары, алюминиевой тары в отдельные контейнеры по видам отходов.

Материал подготовлен на основе информации data.mos.ru

In [72]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=23148")
csvname = unzip(filename)

In [75]:
ecology_trash = pd.read_csv(csvname, sep=';', encoding='cp1251')
ecology_trash = ecology_trash.drop(index=0)
ecology_trash = ecology_trash.drop(columns=[col for col in ecology_trash.columns if col.startswith("Unnamed:")])
ecology_trash.head()

,Code,YardName,global_id,YardType,YardAddress,YardArea,Coordinates,EntryState,EntryAddReason,EntryModifyReason,EntryDeleteReason,geoData,geodata_center,geoarea
1,306306000026,Контейнерная площадка № 3063060,1026288470,контейнерная площадка,"Юго-Западный административный округ Обручевский район город Москва, Ленинский проспект, дом 99 12948",12,"37.521087,55.669857",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.521087,55.669857],""type"":""Point""}","{""coordinates"":[37.521087,55.669857],""type"":""Point""}",NaN
2,308169800026,Контейнерная площадка № 3081698,1026288472,контейнерная площадка,"Юго-Западный административный округ Обручевский район город Москва, улица Академика Челомея, дом 8, корпус 2 28373",12,"37.528221,55.663339",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.528221,55.663339],""type"":""Point""}","{""coordinates"":[37.528221,55.663339],""type"":""Point""}",NaN
3,309294000026,Контейнерная площадка № 3092940,1026288474,контейнерная площадка,"Юго-Западный административный округ Обручевский район город Москва, улица Островитянова, дом 9 280041",12,"37.496381,55.644975",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.496381,55.644975],""type"":""Point""}","{""coordinates"":[37.496381,55.644975],""type"":""Point""}",NaN
4,3584679200026,Площадка для выкатных контейнеров № 35846792,1026288477,площадка для выкатных контейнеров,"Юго-Восточный административный округ район Марьино город Москва, Батайский проезд, дом 51 31446",NaN,"37.738821,55.643411",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.738821,55.643411],""type"":""Point""}","{""coordinates"":[37.738821,55.643411],""type"":""Point""}",NaN
5,3588943600026,Площадка для выкатных контейнеров № 35889436,1026288480,площадка для выкатных контейнеров,"Восточный административный округ район Метрогородок город Москва, улица Николая Химушина, дом 23 64031",NaN,"37.766889,55.825612",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.766889,55.825612],""type"":""Point""}","{""coordinates"":[37.766889,55.825612],""type"":""Point""}",NaN


In [76]:
os.remove(filename)
os.remove(csvname)

In [77]:
ecology_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23159 entries, 1 to 23159
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Code               23159 non-null  object
 1   YardName           17934 non-null  object
 2   global_id          23159 non-null  object
 3   YardType           17934 non-null  object
 4   YardAddress        23159 non-null  object
 5   YardArea           13737 non-null  object
 6   Coordinates        23159 non-null  object
 7   EntryState         23159 non-null  object
 8   EntryAddReason     23159 non-null  object
 9   EntryModifyReason  0 non-null      object
 10  EntryDeleteReason  0 non-null      object
 11  geoData            23159 non-null  object
 12  geodata_center     23159 non-null  object
 13  geoarea            0 non-null      object
dtypes: object(14)
memory usage: 2.5+ MB


## Датасет промышленный предприятий

[Страница](https://data.mos.ru/opendata/2601)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=215363)

Набор данных  «Данные по промышленным предприятиям» содержит подробную информацию об основных крупных и средних предприятиях города Москвы в сфере обрабатывающей промышленности, имеющих производственную локализацию на территории города Москвы. 
Промышленные предприятия являются важнейшим источником пополнения городского бюджета налоговыми отчислениями, создают новые высокопроизводительные рабочие места, обеспечивают приток инвестиций в создание конкурентоспособной промышленной продукции.
Закон г. Москвы от 07.10.2015 N 55 «О промышленной политике города Москвы» определяет цели и задачи промышленной политики города Москвы, устанавливает правовые и экономические основы создания условий для эффективного развития промышленности на территории города Москвы, регулирует отношения, возникающие между субъектами деятельности в сфере промышленности, органами государственной власти города Москвы при формировании и реализации промышленной политики.
 В целях расширения и упорядочения мер государственной поддержки в отношении новых инвестиционных проектов и действующих промышленных предприятий принят Закон города Москвы от 07.10.2015 № 54 «Об инвестиционной политике города Москвы и государственной поддержке субъектов инвестиционной деятельности».
Основные мероприятия по поддержке промышленных предприятий города реализуются в рамках подпрограммы «Москва - город для бизнеса и инноваций» Государственной программы «Экономическое развитие и инвестиционная привлекательность города Москвы» (Постановление Правительства Москвы от 11.10.2011 N 477-ПП).
Современная ситуация в России
На обрабатывающие отрасли промышленности приходится более 13% валового регионального продукта Российской Федерации и более 14% от общего числа занятых в экономике страны.
Одним из основных инструментов реализации государственной политики в промышленности являются государственные программы Российской Федерации, ответственным исполнителем которых выступает Минпромторг России: «Развитие промышленности и повышение ее конкурентоспособности», «Развитие авиационной промышленности», «Развитие электронной и радиоэлектронной промышленности» и другие.
В 2014 году был создан Фонд развития промышленности. Фонд предлагает льготные условия софинансирования проектов, направленных на разработку новой высокотехнологичной продукции, техническое перевооружение и создание конкурентоспособных производств на базе наилучших доступных технологий. Для реализации новых промышленных проектов Фонд на конкурсной основе предоставляет целевые займы по ставке 5% годовых сроком до 7 лет в объеме от 50 до 500 млн рублей, стимулируя приток прямых инвестиций в реальный сектор экономики.
Современная ситуация в Москве
Промышленность Москвы представлена более 20 гражданскими отраслями, ведущими из которых являются пищевая, фармацевтическая, электронная, автомобильная, машиностроительная и другие отрасли.
Обрабатывающие отрасли промышленности играют важную роль в экономике города, обеспечивая более 12% в структуре валового регионального продукта, 7% налоговых поступлений в бюджет города Москвы и 7% от общего числа занятых в экономике города.
Для действующих промышленных предприятий города Москвы Правительством Москвы предоставляются следующие меры поддержки:
1. Присвоение статуса промышленного комплекса.
Порядок получения статуса регламентирован постановлением Правительства Москвы от 11.02.2016 № 38-ПП  «О мерах по реализации промышленной и инвестиционной политики в городе Москве». Присвоение статуса позволяет получать предприятиям региональные налоговые льготы по налогу на прибыль, налогу на имущество, земельному налогу и льготы по аренде земли. Критерии, в соответствии с которыми присваиваются статусы, связаны с эффективностью использования земельных участков, производственных площадей, экономической деятельностью промышленного комплекса, в том числе уровнем заработной платы сотрудников промышленных предприятий.
2. Предоставление субсидий на поддержку организаций, осуществляющих производственную деятельность.
В рамках постановления Правительства Москвы от 20.04.2012 № 152-ПП «Об утверждении порядков предоставления субсидий из бюджета города Москвы организациям, осуществляющим на территории города Москвы деятельность в сфере промышленности» промышленным предприятиям предоставляются субсидии по двум направлениям:
- возмещение части затрат на уплату процентов по кредитам, полученным на приобретение отдельных видов производственного оборудования;
- возмещение части затрат на уплату лизинговых платежей по договорам финансовой аренды (лизинга) отдельных видов производственного оборудования.
 А знаете ли вы что?
Снижение налоговой нагрузки для предприятий, получивших статус промышленного комплекса, может составлять до 17%. Подробную информацию о мерах поддержки промышленности Москвы можно узнать на сайте Департамента науки, промышленной политики и предпринимательства города Москвы в разделе «Промышленность Москвы». (http://dnpp.mos.ru/).
Материал подготовлен на основе информации http://dnpp.mos.ru, http://frprf.ru, http://minpromtorg.gov.ru, http://www.consultant.ru, http://data.mos.ru, http://moscow.gks.ru/, http://www.gks.ru/.

In [78]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=215363")
csvname = unzip(filename)

In [79]:
ecology_prom = pd.read_csv(csvname, sep=';')
ecology_prom = ecology_prom.drop(index=0)
ecology_prom = ecology_prom.drop(columns=[col for col in ecology_prom.columns if col.startswith("Unnamed:")])
ecology_prom.head()

,global_id,ID,FullName,ShortName,INN,OKPO,AdmArea,District,Address,Category,Specialization,OKVED,OKVED_Description,geoData,geodata_center
1,169195889,1,Открытое акционерное общество «ИКМА»,ОАО «ИКМА»,7711037272,11306676,Северный административный округ,район Западное Дегунино,"Ижорская улица, дом 3",Пищевая промышленность,Пищевая,46.32,Торговля оптовая мясом и мясными продуктами,"{coordinates=[37.500640518, 55.883021186], type=Point}",NaN
2,169195890,2,Закрытое акционерное общество «ВЕГУС»,ЗАО «ВЕГУС»,7719245080,58538069,Восточный административный округ,район Соколиная Гора,"Мироновская улица, дом 33, строение 23",Пищевая промышленность,Пищевая,10.13.1,"Производство соленого, вареного, запеченого, копченого, вяленого и прочего мяса","{coordinates=[37.731938402, 55.791989912], type=Point}",NaN
3,169195891,3,Общество с ограниченной ответственностью «МПЗ«Москворецкий»,ООО «МПЗ«Москворецкий»,7724663372,86499888,Южный административный округ,район Москворечье-Сабурово,"Котляковская улица, дом 3, строение 26",Пищевая промышленность,Пищевая,10.13.1,"Производство соленого, вареного, запеченого, копченого, вяленого и прочего мяса","{coordinates=[37.632880109, 55.644013748], type=Point}",NaN
4,169195892,4,Закрытое акционерное общество «ПАРТНЕР Ф»,ЗАО «ПАРТНЕР Ф»,7709239514,18285635,Южный административный округ,район Царицыно,"Кавказский бульвар, дом 58, строение 15",Пищевая промышленность,Пищевая,10.13.1,"Производство соленого, вареного, запеченого, копченого, вяленого и прочего мяса","{coordinates=[37.631797385, 55.630194867], type=Point}",NaN
5,169195893,5,Общество с ограниченной ответственностью «ММПЗ «КОЛОМЕНСКОЕ»,ООО «ММПЗ «КОЛОМЕНСКОЕ»,7725715993,1847158,Южный административный округ,район Нагатино-Садовники,"проспект Андропова, дом 36, корпус 3, строение 1",Пищевая промышленность,Пищевая,10.13,Производство продукции из мяса убойных животных и мяса птицы,"{coordinates=[37.658626187, 55.675476822], type=Point}",NaN


In [76]:
os.remove(filename)
os.remove(csvname)

In [80]:
ecology_prom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 1 to 840
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   global_id          840 non-null    object
 1   ID                 840 non-null    object
 2   FullName           840 non-null    object
 3   ShortName          840 non-null    object
 4   INN                840 non-null    object
 5   OKPO               840 non-null    object
 6   AdmArea            840 non-null    object
 7   District           840 non-null    object
 8   Address            840 non-null    object
 9   Category           840 non-null    object
 10  Specialization     484 non-null    object
 11  OKVED              840 non-null    object
 12  OKVED_Description  840 non-null    object
 13  geoData            840 non-null    object
 14  geodata_center     0 non-null      object
dtypes: object(15)
memory usage: 98.6+ KB


## Датасет рынков

[Страница](https://data.mos.ru/opendata/654)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=206400)

Каталог содержит информацию о розничных рынках, организованных на территории города Москвы с учетом их типологии (27 розничных рынков, из которых: 22 специализированных сельскохозяйственных рынка; 4 универсальных рынка и 1 радиорынок).

В каталогах отражена информация о фактическом месте расположения рынка с указание административного округа, района, адреса размещения, юридический адресе Управляющей рынком компании, ее наименование и специализация рынка.

Сеть розничных рынков города Москвы является важным механизмом для реализации поставки продовольствия, позволяющая потребителям получать производимую сельскохозяйственную продукцию непосредственно от сельхозпроизводителей минуя звенья посредников.

Существующая сеть из 27 розничных рынков, на которых в настоящий момент предусмотрено более 14 тысяч торговых мест, играет важную роль в обеспечении продовольственными товарами значительного количества предприятий розничной торговли и общественного питания, в том числе объектов мелкорозничной торговли, одновременно являясь поставщиком качественных сельскохозяйственных продуктов.

Развитие форматов торговой деятельности в Москве, который мы видим в последние годы существенно затрагивает и розничные рынки.

Если в период до начала 2000-х годов  задачей деятельности розничных рынков можно было считать преодоление товарного дефицита, то в настоящее время розничные рынки эволюционируют в современные инфраструктурные объекты, удобные и безопасные для жителей.

В настоящее время розничные рынки эволюционируют в современные инфраструктурные объекты, удобные и безопасные для жителей.

В связи с вступлением в силу норм Федерального закона от 30.12.2006 № 271-ФЗ «О розничных рынках и о внесении изменений в Трудовой кодекс Российской Федерации», с 01.01.2016 г. торговая деятельность вне капитальных зданий прекращена. Правительство Москвы реализует программу модернизации традиционных московских рынков, для обеспечения москвичей качественной сельскохяйственной продукцией.

Следуя требованиям законодательства, управляющие рынками компании ведут работы по проектированию и строительству капитальных торговых объектов, которые будут построены на месте некапитальных рынков. В этих торговых объектах также будут реализовываться сельскохозяйственные продукты, и покупатели вновь смогут приобретать полюбившиеся им товары, но уже в удобных,  комфортных условиях, в объектах, отвечающих санитарно-техническим нормам и требованиям безопасности. В Зеленограде на месте одноэтажного контейнерного рынка, расположенного на Крюковской площади, уже пострен новый совеременнный многоэтажный сельскохозяйственный рынок, а в ближайшее вемя управляющей компанией на смежных участках будут построены торговый центр и здание ресторана.

Кроме того, капитальные сельскохозяйственные рынки следуя тенденциям торговли также эволюционируют и становятся современнее и удобнее. Так например, проведены работы по реконструкции, Велозаводского и Москворецкого рынков.

А знаете ли вы что?

В Москве существуют рынки со сложившимся историческим обликом, являющимся неотъемлемой частью архитектурного ансамбля города, при их модернизации собственники стараются сохранить внешний облик наиболее значимых объектов.

Например, на одном из старейших и известнейших рынков города  Москвы, Даниловском рынке, проведены масштабные работы по его реконструкции.

При подготовке материалов использован сайт: mos.ru


In [81]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=23148")
csvname = unzip(filename)

In [83]:
ecology_market = pd.read_csv(csvname, sep=';', encoding='cp1251')
ecology_market = ecology_market.drop(index=0)
ecology_market = ecology_market.drop(columns=[col for col in ecology_market.columns if col.startswith("Unnamed:")])
ecology_market.head()

,Code,YardName,global_id,YardType,YardAddress,YardArea,Coordinates,EntryState,EntryAddReason,EntryModifyReason,EntryDeleteReason,geoData,geodata_center,geoarea
1,306306000026,Контейнерная площадка № 3063060,1026288470,контейнерная площадка,"Юго-Западный административный округ Обручевский район город Москва, Ленинский проспект, дом 99 12948",12,"37.521087,55.669857",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.521087,55.669857],""type"":""Point""}","{""coordinates"":[37.521087,55.669857],""type"":""Point""}",NaN
2,308169800026,Контейнерная площадка № 3081698,1026288472,контейнерная площадка,"Юго-Западный административный округ Обручевский район город Москва, улица Академика Челомея, дом 8, корпус 2 28373",12,"37.528221,55.663339",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.528221,55.663339],""type"":""Point""}","{""coordinates"":[37.528221,55.663339],""type"":""Point""}",NaN
3,309294000026,Контейнерная площадка № 3092940,1026288474,контейнерная площадка,"Юго-Западный административный округ Обручевский район город Москва, улица Островитянова, дом 9 280041",12,"37.496381,55.644975",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.496381,55.644975],""type"":""Point""}","{""coordinates"":[37.496381,55.644975],""type"":""Point""}",NaN
4,3584679200026,Площадка для выкатных контейнеров № 35846792,1026288477,площадка для выкатных контейнеров,"Юго-Восточный административный округ район Марьино город Москва, Батайский проезд, дом 51 31446",NaN,"37.738821,55.643411",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.738821,55.643411],""type"":""Point""}","{""coordinates"":[37.738821,55.643411],""type"":""Point""}",NaN
5,3588943600026,Площадка для выкатных контейнеров № 35889436,1026288480,площадка для выкатных контейнеров,"Восточный административный округ район Метрогородок город Москва, улица Николая Химушина, дом 23 64031",NaN,"37.766889,55.825612",активна,Новый объект,NaN,NaN,"{""coordinates"":[37.766889,55.825612],""type"":""Point""}","{""coordinates"":[37.766889,55.825612],""type"":""Point""}",NaN


In [76]:
os.remove(filename)
os.remove(csvname)

In [85]:
ecology_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23159 entries, 1 to 23159
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Code               23159 non-null  object
 1   YardName           17934 non-null  object
 2   global_id          23159 non-null  object
 3   YardType           17934 non-null  object
 4   YardAddress        23159 non-null  object
 5   YardArea           13737 non-null  object
 6   Coordinates        23159 non-null  object
 7   EntryState         23159 non-null  object
 8   EntryAddReason     23159 non-null  object
 9   EntryModifyReason  0 non-null      object
 10  EntryDeleteReason  0 non-null      object
 11  geoData            23159 non-null  object
 12  geodata_center     23159 non-null  object
 13  geoarea            0 non-null      object
dtypes: object(14)
memory usage: 2.5+ MB


## Датасет автовокзалов и автостанций

[страница](https://data.mos.ru/opendata/1881)

[датасет](https://data.mos.ru/odata/export/catalog?idFile=203878)

Набор данных содержит подробную информацию об остановочных пунктах, которые используются для обслуживания межрегиональных и международных автобусных маршрутов.

Автовокзалы и автостанции открыты с целью упорядочить движение междугородного транспорта по городу для удобства москвичей и гостей столицы, обеспечения безопасности перевозок, повышения культуры и качества обслуживания пассажиров, создания цивилизованного рынка транспортных услуг.

Наибольшее число автовокзалов и автостанций сконцентрировано в Южном административном округе:

Работа московских автостанций и автовокзалов регламентирована постановлением Правительства города Москвы от 17.12.2015 N 895-ПП «Об установлении в границах города Москвы остановочных пунктов, которые разрешается использовать в качестве начальных и (или) конечных остановочных пунктов по межрегиональным маршрутам регулярных перевозок, о внесении изменения в постановление Правительства Москвы от 15 февраля 2011 г. N 32-ПП и признании утратившими силу правовых актов города Москвы»

Московские автостанции оборудованы перронами для посадки и высадки пассажиров, кассами по продаже билетов, залами ожидания рейса, местами кратковременной стоянки автобусов. Водители проходят предрейсовый медицинский осмотр, автобусы – технический. Пассажирам предоставляется визуальное и звуковое информационное обеспечение, схемы автобусных маршрутов, расписание отправления и прибытия, таблицы стоимости проезда и провоза багажа.

Современная ситуация в России

Автобусный транспорт широко востребован населением России, он перевозит более 11,5 млрд человек ежегодно, пассажирооборот более 120 млрд. пассажиро-километров.

Доля автобусного транспорта в структуре междугородных перевозок составляет более 30 процентов, уступая только железнодорожному. В России действует более 3000 междугородных маршрутов, выполняется более 3 млн. рейсов ежегодно.

Основная проблема рынка: существенная часть междугородных автобусных перевозок выполняется под видом заказных, где требования транспортной безопасности зачастую не выполняются, перевозчики меняют маршруты и время отправления, а также транспортные средства в пути следования, что справедливо вызывает нарекания пассажиров.

Современная ситуация в Москве

Для удобства москвичей и гостей столицы, обеспечения безопасности перевозок, повышения культуры и качества обслуживания пассажиров, создания цивилизованного рынка транспортных услуг в Москве организованы автовокзалы и автостанции для регулярных межрегиональных и международных автобусных маршрутов.

Большей частью автостанций и автовокзалов управляет государственное унитарное предприятие города Москвы «Мосгортранс». Расположение автостанций и автовокзалов на конечных станциях городского наземного транспорта, вблизи вылетных автомагистралей способствует решению проблемы загруженности центральной части города автобусами, прибывающими в столицу из регионов России и зарубежных стран. Близость к станциям метро делает автостанции удобными для прибытия и отправления пассажиров. А названия, соответствующие ближайшим станциям метрополитена, позволяют легко их найти.

А знаете ли вы что?

Приобрести билет на междугородные и международные автобусы с отправлением из Москвы теперь можно не только в кассах автовокзалов и автостанций, но и в более чем 530 терминалах самообслуживания в Москве и Московской области, адреса расположения терминалов на сайте www.av.mos.ru

In [86]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=203878")
csvname = unzip(filename)

In [88]:
ecology_auto = pd.read_csv(csvname, sep=';')
ecology_auto = ecology_auto.drop(index=0)
ecology_auto = ecology_auto.drop(columns=[col for col in ecology_auto.columns if col.startswith("Unnamed:")])
ecology_auto.head()

,global_id,ID,Name,AdmArea,District,Address,MetroStation,MetroLine,WiFiAvailability,BabyCareRoom,SanitationFacilities,PublicPhone,WebSite,WorkingHours,BusDestination,LocationComment,Latitude_WGS84,Longitude_WGS84,geoData,geodata_center
1,14172647,1,Автостанция «Варшавская»,Южный административный округ,Нагорный район,"город Москва, Каширский проезд, владение 19",Каширская,Замоскворецкая линия,есть,есть,9,nested data,mosmetro.ru,nested data,"Республика Дагестан, Республика Ингушетия, Республика Калмыкия, Республика Крым, Республика Марий Эл, Республика Мордовия, Чеченская Республика, Калужская область, Камчатский край, Краснодарский край, Ставропольский край, Белгородская область, Волгоградская область, Воронежская область, Курская область, Липецкая область, Орловская область, Пензенская область, Ростовская область, Самарская область, Саратовская область, Тамбовская область, Тульская область, город Севастополь, Кабардино - Балкарская Республика, Республика Северная Осетия - Алания, Донецкая Народная Республика, Луганская Народная Республика, Херсонская область, Запорожская область","От метро «Варшавская»: последний вагон, если ехать от станции метро «Каширская». Выход из метро в сторону пригородных поездов. После выхода из вестибюля метро, не поднимаясь наверх, спуститься и пройти по подземному переходу под железной дорогой в сторону Каширского проезда. Выход из подземного перехода налево. После подъёма на улицу повернуть направо. На Каширском проезде повернуть налево, пройти около 300 метров до наземного пешеходного перехода. Далее перейти проезжую часть и вернуться по Каширскому проезду около 70 метров до дома 19.",55.656216,37.624292,"{coordinates=[37.624292, 55.656216], type=Point}",NaN
2,14172681,3,Автостанция «Новоясеневская»,Юго-Западный административный округ,район Ясенево,"город Москва, Новоясеневский тупик, владение 4",Новоясеневская,Калужско-Рижская линия,нет,есть,есть,nested data,busnovoyas.ru,nested data,"Белгородская область, Воронежская область, Калужская область, Липецкая область, Орловская область, Пензенская область, Рязанская область, Тульская область, Донецкая Народная Республика, Луганская Народная Республика, Херсонская область, Запорожская область","Станция метро «Новоясеневская», первый вагон из центра. Станция метро «Битцевский Парк», последний вагон из центра.",55.598421,37.554223,"{coordinates=[37.554223, 55.598421], type=Point}",NaN
3,14172683,7,Автовокзал «Центральный»,Восточный административный округ,район Гольяново,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Гольяново, Щёлковское шоссе, дом 75",Щёлковская,Арбатско-Покровская линия,есть,есть,есть,nested data,mosmetro.ru,nested data,"Республика Башкортостан, Республика Марий Эл, Республика Татарстан (Татарстан), Чувашская Республика - Чувашия, Владимирская область, Вологодская область, Ивановская область, Кировская область, Костромская область, Курганская область, Нижегородская область, Оренбургская область, Пензенская область, Рязанская область, Самарская область, Саратовская область, Тамбовская область, Ульяновская область, Челябинская область, Ярославская область",От метро «Щёлковская»:Первый вагон из центра. В метро следуйте навигационным указателям «Автовокзал». Выход в город № 3. Пройдите вдоль здания ТРЦ «Щёлковский» по соответствующим указателям.,55.811108,37.8000853,"{coordinates=[37.8000853, 55.811108], type=Point}",NaN
4,14172711,9,Международный автовокзал «Южные ворота»,Южный административный округ,район Братеево,"город Москва, 19-й километр Московской Кольцевой Автодороги, владение 20, строение 2",Алма-Атинская,Замоскворецкая линия,есть,есть,13,nested data,mosmetro.ru,nested data,"Республика Адыгея (Адыгея), Республика Алтай, Республика Башкортостан, Республика Бурятия, Республика Дагестан, Республика Ингушетия, Кабардино-Балкарская Республика, Республика Калмыкия, Карачаево-Черкесская Республика, Республика Крым, Республика Марий Эл, Республика Мордовия, Республика Саха (Якутия), Республика Северн

In [89]:
os.remove(filename)
os.remove(csvname)

In [92]:
ecology_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 1 to 8
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   global_id             8 non-null      object
 1   ID                    8 non-null      object
 2   Name                  8 non-null      object
 3   AdmArea               8 non-null      object
 4   District              8 non-null      object
 5   Address               8 non-null      object
 6   MetroStation          8 non-null      object
 7   MetroLine             8 non-null      object
 8   WiFiAvailability      8 non-null      object
 9   BabyCareRoom          8 non-null      object
 10  SanitationFacilities  8 non-null      object
 11  PublicPhone           8 non-null      object
 12  WebSite               8 non-null      object
 13  WorkingHours          8 non-null      object
 14  BusDestination        8 non-null      object
 15  LocationComment       8 non-null      object

## Датасет некачественных автозаправок

https://data.mos.ru/opendata/754/

https://data.mos.ru/odata/export/catalog?idFile=215808

Набор данных содержит информацию о результатах контрольных мероприятий, проведённых Департаментом природопользования и охраны окружающей среды города Москвы в отношении автозаправочных станций с последующими лабораторными испытаниями отобранных на АЗС проб моторного топлива.

Экологические требования к качеству моторного топлива на территории города Москвы установлены постановлением Правительства Москвы от 28.12.2004 № 952-ПП «Об утверждении экологических требований к качеству моторного топлива при его обороте в городе Москве», также данным документом установлен перечень контролируемых Департаментом показателей моторного топлива (Приложение 3 к постановлению Правительства Москвы от 28.12.2004 № 952-ПП).

Перечень основных показателей моторного топлива, контролируемых Департаментом:


1. Автомобильные бензины:

– октановое число по исследовательскому методу;

– октановое число по моторному методу;

– концентрация свинца;

– концентрация серы;

– объёмная доля бензола;

- объёмная доля ароматических углеводородов.

2. Топливо дизельное:

– массовая доля полициклических ароматических углеводородов;

– концентрация серы;

- смазывающая способность;

- общее загрязнение.

Дата-сет позволяет получить подробную информацию об объектах реализации моторного топлива (АЗС), посмотреть их местоположение на карте, узнать  дату и результаты контрольных мероприятий. По каждой проверенной Департаментом АЗС можно посмотреть полное наименование юридического лица, осуществляющего деятельность по реализации моторного топлива на момент контрольных мероприятий, точный адрес АЗС.


Современная ситуация в России

Решением Комиссии Таможенного союза от 2 декабря 2015 года № 84 о внесении изменений в технический регламент таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и мазуту» (ТР ТС 013/2011, на территории Российской Федерации с 1 января 2016 года запрещен выпуск в обращение и обращение дизельного топлива экологического класса Евро-4, а с 1 июля 2016 года выпуск в обращение и обращение автомобильного бензина экологического класса Евро-4.

Постановлением Правительства Российской Федерации от 29 декабря 2012 года        № 1474 «О применении некоторых актов Правительства Российской Федерации по вопросам технического регулирования и об органах государственного контроля (надзора) за соблюдением требований технического регламента таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и топочному мазуту» установлено, что: Федеральное агентство по техническому регулированию и метрологии осуществляет государственный контроль (надзор) за соблюдением требований к автомобильному бензину, дизельному топливу, судовому топливу и мазуту, установленных Техническим регламентом Таможенного союза.


Современная ситуация по реализации моторного топлива в городе Москве


По итогам 2016 года в результате контрольных мероприятий на АЗС города Москвы выявлено 11 проб (11,2% от общего количества отобранных проб) моторного топлива, не соответствует утвержденным экологическим требованиям, в том числе 8 проб автомобильного бензина и 3 пробы дизельного топлива.

По данным фактам Департаментом виновные лица привлечены к административной ответственности в виде административных штрафов, также виновным лицам выданы предписания на устранение нарушений. В случае невыполнения выданных предписаний виновные лица также привлекаются Департаментом к административной ответственности в виде административных штрафов.

А знаете ли вы, что...?

- АЗС на территории города Москвы контролируются Департаментом на предмет соответствия реализуемого моторного топлива экологическим требованиям с 2004 года в границах города до 1 июля 2012 года и с 1 июля 2012 года в современных границах города Москвы.

- В случае выявления фактов реализации на АЗС моторного топлива, не соответствующего экологическим требованиям, установленным на территории города Москвы, Вы вправе обратиться в Единую справочную службу Мэрии Москвы по тел.    8 (495) 777-77-77  либо письменно в Департамент (119019, Москва, ул. Новый Арбат, д. 11, корп. 1 либо через портал Департамента (http://eco.mos.ru/) с просьбой провести контрольные мероприятия в отношении указанных АЗС. При этом в обращении требуется указать причины обращения (перечислить признаки реализации моторного топлива, не соответствующего установленным экологическим требованиям), точный адрес расположения АЗС на территории города Москвы, Фамилию, Имя, Отчество и контактные данные (адрес заявителя и телефонный номер).

- В соответствии с положениями статьи 4.10 (Производство, оборот нефтепродуктов, не соответствующих экологическим требованиям) Кодекса города Москвы об административных правонарушениях установлена административная ответственность в виде административного штрафа на лиц, осуществляющих предпринимательскую деятельность без образования юридического лица, в размере 50 тысяч рублей; на юридических лиц – 400 тысяч рублей. При этом повторное совершение указанного административного правонарушения влечет наложение административного штрафа на юридических лиц в размере одного миллиона рублей.

Материал подготовлен на основе информации, полученной в результате контрольных мероприятий, проведённых Департаментом, и нормативных правовых актов Российской Федерации и города Москвы, http://data.mos.ru/.

In [93]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=215808")
csvname = unzip(filename)

In [94]:
ecology_bad_gas = pd.read_csv(csvname, sep=';')
ecology_bad_gas = ecology_bad_gas.drop(index=0)
ecology_bad_gas = ecology_bad_gas.drop(columns=[col for col in ecology_bad_gas.columns if col.startswith("Unnamed:")])
ecology_bad_gas.head()

,global_id,ID,FullName,ShortName,AdmArea,District,Address,Owner,Violations,TestDate,geoData,geodata_center,Unnamed: 12
1,25792131,16,Автомобильная заправочная станция «ДЭКа-ОЙЛ» на 58-м километре Калужского шоссе,Автомобильная заправочная станция «ДЭКа-ОЙЛ» на 58-м километре Калужского шоссе,Троицкий административный округ,поселение Вороновское,58-й километр Калужского шоссе,ООО «ДЭКа-ОЙЛ»,nested data,14.05.2015,"{coordinates=[37.172439523, 55.343229037], type=Point}","{coordinates=[37.172439523, 55.343229037], type=Point}",NaN
2,25797283,17,Автомобильная заправочная станция «НефтеХим Нафта» на 41-м километре Калужского шоссе,АЗС «НефтеХим Нафта» на 41-м километре Калужского шоссе,Троицкий административный округ,поселение Троицк,41-й километр Калужского шоссе,ООО «НефтеХим Нафта»,nested data,19.05.2015,"{coordinates=[37.299622258, 55.466529149], type=Point}","{coordinates=[37.299622258, 55.466529149], type=Point}",NaN
3,25797951,18,"Автомобильная заправочная станция «Зинкин А.Д.» в Зеленограде, на проезде 4807, строение 6","АЗС «Зинкин А.Д.» в Зеленограде, на проезде 4807, строение 6",Зеленоградский административный округ,район Савёлки,"Зеленоград, проезд 4807, строение 6",ИП Зинкин А.Д.,nested data,20.05.2015,"{coordinates=[37.257993959, 55.979719171], type=Point}","{coordinates=[37.257993959, 55.979719171], type=Point}",NaN
4,25799035,19,"Нефтебаза «Нефтепродуктсервис» в деревне Старосырово, на Симферопольском шоссе, дом 20, строение 1","НБ «Нефтепродуктсервис» в деревне Старосырово, на Симферопольском шоссе, дом 20, строение 1",Новомосковский административный округ,поселение Рязановское,"деревня Старосырово, Симферопольское шоссе, дом 20, строение 1",ООО «Нефтепродуктсервис»,nested data,21.05.2015,"{coordinates=[37.537591224, 55.486356283], type=Point}","{coordinates=[37.537591224, 55.486356283], type=Point}",NaN
5,25800062,20,"Автомобильная заправочная станция «Ресурс-Трейдинг» на Южнопортовой улице, дом 17Д","АЗС «Ресурс-Трейдинг» на Южнопортовой улице, дом 17Д",Юго-Восточный административный округ,район Печатники,"Южнопортовая улица, дом 17Д",ООО «Ресурс-Трейдинг»,nested data,26.05.2015,"{coordinates=[37.702890764, 55.704308594], type=Point}","{coordinates=[37.702890764, 55.704308594], type=Point}",NaN


In [95]:
os.remove(filename)
os.remove(csvname)

In [97]:
ecology_bad_gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 1 to 16
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   global_id       16 non-null     object 
 1   ID              16 non-null     object 
 2   FullName        16 non-null     object 
 3   ShortName       16 non-null     object 
 4   AdmArea         16 non-null     object 
 5   District        16 non-null     object 
 6   Address         16 non-null     object 
 7   Owner           16 non-null     object 
 8   Violations      16 non-null     object 
 9   TestDate        16 non-null     object 
 10  geoData         16 non-null     object 
 11  geodata_center  16 non-null     object 
 12  Unnamed: 12     0 non-null      float64
dtypes: float64(1), object(12)
memory usage: 1.8+ KB


## Датасет автозапровок

https://data.mos.ru/opendata/753

https://data.mos.ru/odata/export/catalog?idFile=215812

Набор данных содержит информацию о результатах контрольных мероприятий, проведённых Департаментом природопользования и охраны окружающей среды города Москвы в отношении автозаправочных станций с последующими лабораторными испытаниями отобранных на АЗС проб моторного топлива.

Экологические требования к качеству моторного топлива на территории города Москвы установлены постановлением Правительства Москвы от 28.12.2004 № 952-ПП «Об утверждении экологических требований к качеству моторного топлива при его обороте в городе Москве», также данным документом установлен перечень контролируемых Департаментом показателей моторного топлива (Приложение 3 к постановлению Правительства Москвы от 28.12.2004 № 952-ПП).

Перечень основных показателей моторного топлива, контролируемых Департаментом:


1. Автомобильные бензины:

– октановое число по исследовательскому методу;

– октановое число по моторному методу;

– концентрация свинца;

– концентрация серы;

– объёмная доля бензола;

- объёмная доля ароматических углеводородов.


2. Топливо дизельное:

– массовая доля полициклических ароматических углеводородов;

– концентрация серы;

- смазывающая способность;

- общее загрязнение.


Дата-сет позволяет получить подробную информацию об объектах реализации моторного топлива (АЗС), посмотреть их местоположение на карте, узнать  дату и результаты контрольных мероприятий. По каждой проверенной Департаментом АЗС можно посмотреть полное наименование юридического лица, осуществляющего деятельность по реализации моторного топлива на момент контрольных мероприятий, точный адрес АЗС.


Современная ситуация в России


Решением Комиссии Таможенного союза от 2 декабря 2015 года № 84 о внесении изменений в технический регламент таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и мазуту» (ТР ТС 013/2011, на территории Российской Федерации с 1 января 2016 года запрещен выпуск в обращение и обращение дизельного топлива экологического класса Евро-4, а с 1 июля 2016 года выпуск в обращение и обращение автомобильного бензина экологического класса Евро-4.


Постановлением Правительства Российской Федерации от 29 декабря 2012 года        № 1474 «О применении некоторых актов Правительства Российской Федерации по вопросам технического регулирования и об органах государственного контроля (надзора) за соблюдением требований технического регламента таможенного союза «О требованиях к автомобильному и авиационному бензину, дизельному и судовому топливу, топливу для реактивных двигателей и топочному мазуту» установлено, что: Федеральное агентство по техническому регулированию и метрологии осуществляет государственный контроль (надзор) за соблюдением требований к автомобильному бензину, дизельному топливу, судовому топливу и мазуту, установленных Техническим регламентом Таможенного союза.


Современная ситуация по реализации моторного топлива в городе Москве

По итогам 2016 года в результате контрольных мероприятий на АЗС города Москвы выявлено 11 проб (11,2% от общего количества отобранных проб) моторного топлива, не соответствует утвержденным экологическим требованиям, в том числе     8 проб автомобильного бензина и 3 пробы дизельного топлива.

По данным фактам Департаментом виновные лица привлечены к административной ответственности в виде административных штрафов, также виновным лицам выданы предписания на устранение нарушений. В случае невыполнения выданных предписаний виновные лица также привлекаются Департаментом к административной ответственности в виде административных штрафов.

А знаете ли вы, что...?

- АЗС на территории города Москвы контролируются Департаментом на предмет соответствия реализуемого моторного топлива экологическим требованиям с 2004 года в границах города до 1 июля 2012 года и с 1 июля 2012 года в современных границах города Москвы.

- В случае выявления фактов реализации на АЗС моторного топлива, не соответствующего экологическим требованиям, установленным на территории города Москвы, Вы вправе обратиться в Единую справочную службу Мэрии Москвы по тел.    8 (495) 777-77-77 либо письменно в Департамент (119019, Москва, ул. Новый Арбат,  д. 11, корп. 1 либо через портал Департамента (http://eco.mos.ru/) с просьбой провести контрольные мероприятия в отношении указанных АЗС. При этом в обращении требуется указать причины обращения (перечислить признаки реализации моторного топлива, не соответствующего установленным экологическим требованиям), точный адрес расположения АЗС на территории города Москвы, Фамилию, Имя, Отчество и контактные данные (адрес заявителя и телефонный номер).

- В соответствии с положениями статьи 4.10 (Производство, оборот нефтепродуктов, не соответствующих экологическим требованиям) Кодекса города Москвы об административных правонарушениях установлена административная ответственность в виде административного штрафа на лиц, осуществляющих предпринимательскую деятельность без образования юридического лица, в размере 50 тысяч рублей; на юридических лиц – 400 тысяч рублей. При этом повторное совершение указанного административного правонарушения влечет наложение административного штрафа на юридических лиц в размере одного миллиона рублей.

Материал подготовлен на основе информации, полученной в результате контрольных мероприятий, проведённых Департаментом, и нормативных правовых актов Российской Федерации и города Москвы, http://data.mos.ru/.

In [98]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=215812")
csvname = unzip(filename)

In [100]:
ecology_gas = pd.read_csv(csvname, sep=';')
ecology_gas = ecology_gas.drop(index=0)
ecology_gas = ecology_gas.drop(columns=[col for col in ecology_gas.columns if col.startswith("Unnamed:")])
ecology_gas.head()

,global_id,ID,FullName,ShortName,AdmArea,District,Address,Owner,TestDate,geoData,geodata_center
1,4539435,14,Автомобильный заправочный комплекс № 1 «РН-Москва»,АЗК № 1 «РН-Москва»,Северо-Восточный административный округ,район Отрадное,"улица Декабристов, дом 47Б",ОАО «РН-Москва»,04.04.2013,"{coordinates=[37.623636836, 55.858348644], type=Point}",NaN
2,4539436,15,Автомобильный заправочный комплекс № 11 «РН-Москва»,АЗК № 11 «РН-Москва»,Северный административный округ,Бескудниковский район,"Дубнинская улица, дом 52",ОАО «РН-Москва»,04.04.2013,"{coordinates=[37.553284632, 55.896976741], type=Point}","{coordinates=[37.553284632, 55.896976741], type=Point}"
3,4539437,16,"Автомобильная заправочная станция «Альфа-Петрол М» на Ясеневой улице, дом 13А","АЗС «Альфа-Петрол М» на Ясеневой улице, дом 13А",Южный административный округ,район Орехово-Борисово Южное,"Ясеневая улица, дом 13А",ООО «Альфа-Петрол М»,09.04.2013,"{coordinates=[37.725173521, 55.602283341], type=Point}",NaN
4,4539438,19,Автомобильная заправочная станция № 18 «ЕКА-АЗС»,АЗС № 18 «ЕКА-АЗС»,Северный административный округ,район Сокол,"улица Панфилова, владение 2",ООО «ЕКА-АЗС»,10.04.2013,"{coordinates=[37.500852903, 55.79859734], type=Point}",NaN
5,4539439,20,Автомобильный заправочный комплекс № 57 «РН-Москва»,АЗК № 57 «РН-Москва»,Юго-Восточный административный округ,Рязанский район,"Рязанский проспект, владение 26, корпус 2",ОАО «РН-Москва»,11.04.2013,"{coordinates=[37.777492911, 55.719633634], type=Point}","{coordinates=[37.777492911, 55.719633634], type=Point}"


In [76]:
os.remove(filename)
os.remove(csvname)

In [77]:
ecology_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23159 entries, 1 to 23159
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Code               23159 non-null  object
 1   YardName           17934 non-null  object
 2   global_id          23159 non-null  object
 3   YardType           17934 non-null  object
 4   YardAddress        23159 non-null  object
 5   YardArea           13737 non-null  object
 6   Coordinates        23159 non-null  object
 7   EntryState         23159 non-null  object
 8   EntryAddReason     23159 non-null  object
 9   EntryModifyReason  0 non-null      object
 10  EntryDeleteReason  0 non-null      object
 11  geoData            23159 non-null  object
 12  geodata_center     23159 non-null  object
 13  geoarea            0 non-null      object
dtypes: object(14)
memory usage: 2.5+ MB
